## Humanitarian Relief Project
Prepared by, A. Lamsal and D.O. Oral

## 0. Necessities
Run the below lying code (Without "#") to download all of the necessities at once.

In [ ]:
# !pip install -r requirements.txt

Run the code snippet below once, before starting your work every time. This will allow the code to access configurations.

In [ ]:
# Configuration File Load

import yaml

with open("config.yaml", "r", encoding="utf-8") as _f:
    CFG = yaml.safe_load(_f) or {}

class ConfigError(Exception):
    pass

def require(key, cast=None):
    if key not in CFG:
        raise ConfigError(f"Missing required config key: {key}")
    val = CFG[key]
    try:
        return cast(val) if cast else val
    except Exception as e:
        raise ConfigError(f"Bad type for key '{key}': {val!r} ({e})")

def optional(key, cast=None):
    if key not in CFG:
        return None
    val = CFG[key]
    return cast(val) if cast else val

## A. Data Download and Display

In [ ]:
# =============================== A0) Imports ===============================
import os
import io
import time
from pathlib import Path

import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, box

import osmnx as ox
from owslib.wfs import WebFeatureService

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib_scalebar.scalebar import ScaleBar

from IPython.display import display
import yaml


In [ ]:
# ============== A1) Data, Outputs, and User Parameters ===================

# ---- Directories ----
DATA_DIR   = r"./Data"
OUT_DIR_BASE = r"./Outputs"

# Section-scoped outputs (this is Section A)
OUT_DIR      = os.path.join(OUT_DIR_BASE, "A")
IMAGES_DIR   = os.path.join(OUT_DIR_BASE, "Images")

# Ensure output directories exist
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

# ---- AOI selection mode: choose ONE of "place", "coords", or "shapefile"
MODE = require("MODE", str)       # "place" | "coords" | "shapefile"

# If MODE == "place": the region name to query from OSM
PLACE_NAME = require("PLACE_NAME", str)

# Historical OSM snapshot date (optional in Configuration)
target_dt = require("target_dt", str)
ox.settings.overpass_settings   = f'[out:json][timeout:180][date:"{target_dt}"]'
ox.settings.overpass_rate_limit = True

# WFS server/layer
WFS_URL   = require("WFS_URL", str)
WFS_LAYER = require("WFS_LAYER", str)

# Output filenames (written under OUT_DIR)
OUT_BLDG      = "buildings_selected_region.shp"
OUT_NODES     = "osm_nodes.shp"
OUT_EDGES     = "osm_edges.shp"
OUT_CITY_POLY = "region_boundary.geojson"   # saved when MODE == "place"

# Grid splitting for WFS
GRID_PARTS = require("GRID_PARTS", int)   # e.g., 4x4

# Optional: pad the AOI bounding box used for WFS (degrees)
BBOX_PAD_DEG = require("BBOX_PAD_DEG", float)

# If MODE == "coords": manual AOI polygon coordinates (lon, lat)
if MODE == "coords":
    raw_coords = require("AOI_COORDS")  # expect list of [lon, lat]
    if raw_coords is None:
        raise ValueError("AOI_COORDS must be set when MODE == 'coords'.")
    AOI_COORDS = [tuple(p) for p in raw_coords]
else:
    AOI_COORDS = None  # not used

# If MODE == "shapefile": path to a polygon shapefile (inside DATA_DIR)
if MODE == "shapefile":
    SHP_PATH = os.path.join(DATA_DIR, require("SHP_NAME", str))
else:
    SHP_PATH = None

In [ ]:
# ============================ A2) Helper Functions =========================
def split_polygon(gdf_in: gpd.GeoDataFrame, n_parts: int = 16) -> gpd.GeoDataFrame:
    """
    Make an equal grid over the AOI and intersect to get sub-polygons.
    For n_parts=16, creates a 4x4 grid over the AOI extent.
    """
    if gdf_in.empty:
        return gpd.GeoDataFrame(geometry=[], crs=gdf_in.crs)

    minx, miny, maxx, maxy = gdf_in.total_bounds
    cols = int(n_parts ** 0.5)
    rows = int(n_parts / cols)
    dx = (maxx - minx) / cols
    dy = (maxy - miny) / rows

    cells = []
    for i in range(cols):
        for j in range(rows):
            x1, y1 = minx + i * dx, miny + j * dy
            x2, y2 = x1 + dx, y1 + dy
            cells.append(box(x1, y1, x2, y2))

    grid = gpd.GeoDataFrame(geometry=cells, crs=gdf_in.crs)
    # Intersect grid with AOI to keep only overlapping parts
    try:
        out = gpd.overlay(grid, gdf_in, how="intersection", keep_geom_type=True)
    except Exception:
        out = grid
    return out


def download_buildings_chunk(
    wfs: WebFeatureService,
    bbox_3857: tuple,
    typename: str,
    srsname: str = "urn:x-ogc:def:crs:EPSG:3857",
    max_retries: int = 3,
    wait_sec: int = 5
) -> gpd.GeoDataFrame:
    """
    Download buildings using a rectangular bbox (EPSG:3857).
    """
    for attempt in range(1, max_retries + 1):
        try:
            response = wfs.getfeature(
                typename=typename,
                bbox=bbox_3857,
                srsname=srsname,
                outputFormat="application/json",
            )
            gdf = gpd.read_file(io.BytesIO(response.read()))
            if gdf.crs is None:
                gdf.set_crs("EPSG:3857", inplace=True)
            return gdf
        except Exception as e:
            print(f"  Error on attempt {attempt}/{max_retries} for bbox={bbox_3857}: {e}")
            if attempt < max_retries:
                time.sleep(wait_sec)

    return gpd.GeoDataFrame(geometry=[], crs="EPSG:3857")


In [ ]:
# ===== A3) OSM download ===========
# ===== Warning Suppression =====
import warnings
warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
    message="The 'unary_union' attribute is deprecated, use the 'union_all\\(\\)' method instead."
)
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    message="Column names longer than 10 characters will be truncated when saved to ESRI Shapefile."
)
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    message="Normalized/laundered field name:.*"
)

# Downloading
print(f"[OSM Download Mode] {MODE!r}")

if MODE == "place":
    print(f"Fetching OSM admin polygon for: {PLACE_NAME}")
    region_boundary_gdf = ox.geocode_to_gdf(PLACE_NAME).to_crs("EPSG:4326")

    # Save region boundary polygon
    region_poly_path = os.path.join(OUT_DIR, OUT_CITY_POLY)
    region_boundary_gdf.to_file(region_poly_path, driver="GeoJSON")
    print(f"Saved region boundary → {region_poly_path}")

    # Download OSM network within the exact region polygon
    print("Downloading OSM network within region polygon ...")
    region_geom = region_boundary_gdf.unary_union
    G_city = ox.graph_from_polygon(region_geom, network_type="all")
    nodes, edges = ox.graph_to_gdfs(G_city)

    # AOI polygon for later use in WFS
    aoi_polygon_ll = region_boundary_gdf.unary_union
    aoi_gdf_ll = gpd.GeoDataFrame(geometry=[aoi_polygon_ll], crs="EPSG:4326")

elif MODE == "shapefile":
    print(f"Loading AOI from shapefile: {SHP_PATH}")
    aoi_gdf_ll = gpd.read_file(SHP_PATH).to_crs("EPSG:4326")
    aoi_geom = aoi_gdf_ll.unary_union

    print("Downloading OSM network within AOI polygon ...")
    G_city = ox.graph_from_polygon(aoi_geom, network_type="all")
    nodes, edges = ox.graph_to_gdfs(G_city)

    # Also save the region boundary in this mode:
    region_boundary_gdf = aoi_gdf_ll.copy()
    region_poly_path = os.path.join(OUT_DIR, OUT_CITY_POLY)
    region_boundary_gdf.to_file(region_poly_path, driver="GeoJSON")
    print(f"Saved region boundary (from shapefile) → {region_poly_path}")

elif MODE == "coords":
    print("Building AOI polygon from manual coordinates ...")
    polygon_ll = Polygon(AOI_COORDS)
    aoi_gdf_ll = gpd.GeoDataFrame(geometry=[polygon_ll], crs="EPSG:4326")

    print("Downloading OSM network within AOI polygon ...")
    G_city = ox.graph_from_polygon(aoi_gdf_ll.unary_union, network_type="all")
    nodes, edges = ox.graph_to_gdfs(G_city)

    # Also save the region boundary in this mode:
    region_boundary_gdf = aoi_gdf_ll.copy()
    region_poly_path = os.path.join(OUT_DIR, OUT_CITY_POLY)
    region_boundary_gdf.to_file(region_poly_path, driver="GeoJSON")
    print(f"Saved region boundary (from coords) → {region_poly_path}")

else:
    raise ValueError("MODE must be one of: 'place', 'coords', 'shapefile'")

# Save OSM nodes and edges into Outputs\A
nodes_path = os.path.join(OUT_DIR, OUT_NODES)
edges_path = os.path.join(OUT_DIR, OUT_EDGES)
nodes.to_file(nodes_path)
edges.to_file(edges_path)
print(f"Saved OSM nodes → {nodes_path}")
print(f"Saved OSM edges → {edges_path}")


In [ ]:
# === A4) WFS download using AOI bbox → clip to AOI polygon → save ==========
# Compute lon/lat min/max from AOI polygon
minx, miny, maxx, maxy = aoi_gdf_ll.total_bounds
if BBOX_PAD_DEG and BBOX_PAD_DEG > 0:
    minx -= BBOX_PAD_DEG; miny -= BBOX_PAD_DEG
    maxx += BBOX_PAD_DEG; maxy += BBOX_PAD_DEG

print("AOI bbox (lon/lat):")
print(f"  lon: {minx:.6f} .. {maxx:.6f}")
print(f"  lat: {miny:.6f} .. {maxy:.6f}")

# Convert AOI to EPSG:3857 and split into smaller polygons for WFS
aoi_gdf_3857 = aoi_gdf_ll.to_crs("EPSG:3857")
sub_polys_3857 = split_polygon(aoi_gdf_3857, n_parts=GRID_PARTS)
print(f"AOI split into {len(sub_polys_3857)} parts for WFS.\n")

# Connect to WFS
print("Connecting to WFS ...")
wfs = WebFeatureService(url=WFS_URL, version="1.1.0", timeout=120)
print("Connected.\n")

# Download per grid cell via BBOX
all_chunks = []
for i, row in sub_polys_3857.iterrows():
    part_id = i + 1
    cell_bounds = tuple(row.geometry.bounds)
    print(f"Downloading buildings via WFS: part {part_id}/{len(sub_polys_3857)}")
    gdf_part = download_buildings_chunk(
        wfs=wfs,
        bbox_3857=cell_bounds,
        typename=WFS_LAYER,
        srsname="urn:x-ogc:def:crs:EPSG:3857",
        max_retries=3,
        wait_sec=5
    )
    if gdf_part.empty:
        print(f"  No buildings for part {part_id}.")
        continue

    # Clip to the exact cell polygon
    cell_gdf = gpd.GeoDataFrame(geometry=[row.geometry], crs=sub_polys_3857.crs)
    gdf_clip_cell = gpd.overlay(gdf_part, cell_gdf, how="intersection")
    if not gdf_clip_cell.empty:
        all_chunks.append(gdf_clip_cell)

if not all_chunks:
    raise SystemExit("No buildings were downloaded from WFS for the AOI.")

# Merge and clip to the full AOI polygon
print("\nMerging WFS parts ...")
total_chunks = len(all_chunks)
print(f"  Total non-empty chunks to merge: {total_chunks}")

merged_list = []
for idx, gdf_chunk in enumerate(all_chunks, start=1):
    merged_list.append(gdf_chunk)
    # lightweight "progress bar" in the same line
    print(f"  Merging chunks: {idx}/{total_chunks}", end="\r")

# finalize the progress line
print()

bld_3857_raw = gpd.GeoDataFrame(pd.concat(merged_list, ignore_index=True), crs="EPSG:3857")

# Drop duplicate geometries
before_n = len(bld_3857_raw)
print("  Dropping duplicate geometries ...")
bld_3857_raw = bld_3857_raw.drop_duplicates(subset="geometry")
after_n = len(bld_3857_raw)
removed_n = before_n - after_n
print(f"  Kept {after_n} unique buildings (removed {removed_n} duplicates).")

print("  Clipping merged buildings to full AOI polygon ...")
aoi_full_3857 = aoi_gdf_ll.to_crs("EPSG:3857")
bld_3857 = gpd.overlay(bld_3857_raw, aoi_full_3857, how="intersection")

# Add centroid lon/lat (correct: centroid in projected CRS, then to EPSG:4326)
print("  Computing centroids ...")
centroids_3857 = bld_3857.geometry.centroid          # in EPSG:3857
centroids_ll = centroids_3857.to_crs("EPSG:4326")    # convert to lon/lat

bld_3857["longitude"] = centroids_ll.x
bld_3857["latitude"]  = centroids_ll.y

# Save buildings shapefile into Outputs\A
bldg_path = os.path.join(OUT_DIR, OUT_BLDG)
bld_3857.to_file(bldg_path)
print(f"\nExported AOI-clipped building shapefile → {bldg_path}")
print(f"Total buildings saved (inside AOI): {len(bld_3857)}")


In [ ]:
# ======================= A5) Quick inspection (CRS + samples) ===============
datasets = {
    "OSM_Nodes": globals().get("nodes", None),
    "OSM_Edges": globals().get("edges", None),
    "Region_Boundary": globals().get("region_boundary_gdf", None),  # if available
}

# Prefer previously created buildings if present; otherwise use fresh result
if "buildings_r" in globals() and (globals()["buildings_r"] is not None) and (not globals()["buildings_r"].empty):
    buildings_gdf = globals()["buildings_r"]
elif "bldg" in globals() and (globals()["bldg"] is not None) and (not globals()["bldg"].empty):
    buildings_gdf = globals()["bldg"]
elif "bld_3857" in globals() and (globals()["bld_3857"] is not None) and (not globals()["bld_3857"].empty):
    buildings_gdf = globals()["bld_3857"]
else:
    buildings_gdf = globals().get("bld_3857", None)

datasets["Buildings"] = buildings_gdf

# --- CRS report ---
print(" Coordinate Reference Systems (CRS) of current GeoDataFrames:\n")
for name, gdf in datasets.items():
    if gdf is not None and hasattr(gdf, "crs"):
        print(f"{name:20s}: {gdf.crs}")
    else:
        print(f"{name:20s}: (not found or no CRS assigned)")

# --- Example rows ---
print("\n Example rows from loaded GeoDataFrames:\n")
for name, gdf in datasets.items():
    if gdf is not None and hasattr(gdf, "empty") and (not gdf.empty):
        print(f"── {name} ───────────────────────────────────────────────")
        try:
            display(gdf.drop(columns="geometry").head(5))
        except Exception:
            display(pd.DataFrame(gdf.head(5)))
    else:
        print(f"── {name}: (not found or empty)\n")


In [ ]:
# =============== A6) Plotting  ===============
P = {
    # ===== General Drawing Toggles =====
    "DRAW_EDGES": True,                 # draw OSM road edges
    "DRAW_NODES": False,                # draw OSM network nodes
    "DRAW_BUILDINGS": True,             # draw building footprints
    "DRAW_REGION": True,                # draw AOI boundary polygon

    # ===== Manual Zoom (EPSG:4326) =====
    "lon_min": None,                    # min longitude for custom zoom (None = auto)
    "lon_max": None,                    # max longitude for custom zoom (None = auto)
    "lat_min": None,                    # min latitude for custom zoom (None = auto)
    "lat_max": None,                    # max latitude for custom zoom (None = auto)

    # ===== Figure & Labels =====
    "FIGSIZE": (11, 10),                # figure size (width, height) in inches
    "DPI": 300,                         # export resolution (dots per inch)
    "TITLE": "AOI, Edges and Buildings",  # plot title
    "TITLE_FONTSIZE": 13,               # title font size (pt)
    "XLABEL": "Easting (m)",            # x-axis label (projected meters)
    "YLABEL": "Northing (m)",           # y-axis label (projected meters)

    # ===== Region Boundary Style =====
    "REGION_COLOR": "black",            # region boundary color
    "REGION_LW": 1.2,                   # region boundary line width
    "REGION_ALPHA": 1.0,                # region boundary transparency (0..1)
    "REGION_LABEL": "Region Boundary",  # legend label for region

    # ===== Road / Edge Style =====
    "EDGE_COLOR": "#9e9e9e",            # edge color
    "EDGE_LW": 0.6,                     # edge line width
    "EDGE_ALPHA": 1.0,                  # edge transparency (0..1)
    "EDGE_LABEL": "Edges",              # legend label for edges

    # ===== Building Style =====
    "BLDG_FACE": "black",               # building fill color
    "BLDG_EDGE": "none",                # building outline color ('none' = no outline)
    "BLDG_LW": 0.2,                     # building outline width (if not 'none')
    "BLDG_ALPHA": 1.0,                  # building fill transparency (0..1)
    "BLDG_LABEL": "Buildings",          # legend label for buildings

    # ===== Node Style =====
    "NODE_COLOR": "#1f78b4",            # node color
    "NODE_SIZE": 1,                     # node marker size
    "NODE_ALPHA": 1.0,                  # node transparency (0..1)
    "NODE_MAX_PLOT": 150000,            # sample cap to avoid over-plotting
    "NODE_LABEL": "Nodes",              # legend label for nodes

    # ===== North Arrow =====
    "ADD_NORTH_ARROW": True,            # show north arrow
    "NA_X": 0.08,                       # arrow x-position (axes fraction 0–1)
    "NA_Y": 0.12,                       # arrow y-position (axes fraction 0–1)
    "NA_LEN": 0.08,                     # arrow length (axes fraction)
    "NA_LABEL": "N",                    # arrow text
    "NA_COLOR": "black",                # arrow color
    "NA_LW": 2,                         # arrow line width
    "NA_FONTSIZE": 14,                  # 'N' font size

    # ===== Scalebar =====
    "ADD_SCALEBAR": True,               # show scalebar
    "SB_DX": 1,                         # units-per-pixel (1 when plotting in meters/UTM)
    "SB_UNITS": "m",                    # units label for scalebar
    "SB_LOC": "lower right",            # scalebar location
    "SB_BOX_ALPHA": 0.8,                # scalebar box transparency (0..1)
    "SB_COLOR": "black",                # scalebar text/line color

    # ===== Legend =====
    "SHOW_LEGEND": True,                # show legend
    "LEGEND_LOC": "upper right",        # legend location
    "LEGEND_FRAME": True,               # draw legend frame box
    "LEGEND_FACE": "white",             # legend box facecolor
    "LEGEND_EDGE": "black",             # legend box edgecolor

    # ===== Output =====
    "OUT_NAME": "Region_UTM_Map_Datasets.png"  # output PNG filename
}

# --- minimal helpers ---
def add_north(ax, x, y, length, color, lw, fs, label):
    ax.annotate(label, xy=(x, y), xytext=(x, y - length),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=fs, fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=lw, color=color),
                clip_on=False, zorder=20)

def add_scalebar(ax, dx, units, loc, alpha, color):
    ax.add_artist(ScaleBar(dx, units, location=loc, box_alpha=alpha, color=color))

# --- choose a CRS source ---
cand = None
if P["DRAW_BUILDINGS"] and buildings_gdf is not None and not buildings_gdf.empty:
    cand = buildings_gdf
elif P["DRAW_EDGES"] and datasets["OSM_Edges"] is not None and not datasets["OSM_Edges"].empty:
    cand = datasets["OSM_Edges"]
elif P["DRAW_NODES"] and datasets["OSM_Nodes"] is not None and not datasets["OSM_Nodes"].empty:
    cand = datasets["OSM_Nodes"]
elif P["DRAW_REGION"] and datasets["Region_Boundary"] is not None and not datasets["Region_Boundary"].empty:
    cand = datasets["Region_Boundary"]

utm_crs = cand.to_crs(4326).estimate_utm_crs()

# --- reproject only what we draw ---
edges_utm  = datasets["OSM_Edges"].to_crs(utm_crs)       if P["DRAW_EDGES"]   and datasets["OSM_Edges"]   is not None else None
nodes_utm  = datasets["OSM_Nodes"].to_crs(utm_crs)       if P["DRAW_NODES"]   and datasets["OSM_Nodes"]   is not None else None
bldg_utm   = buildings_gdf.to_crs(utm_crs)               if P["DRAW_BUILDINGS"] and buildings_gdf is not None       else None
region_utm = datasets["Region_Boundary"].to_crs(utm_crs) if P["DRAW_REGION"]  and datasets["Region_Boundary"] is not None else None

# --- optional zoom (EPSG:4326 → UTM bounds) ---
use_zoom = all(v is not None for v in [P["lon_min"], P["lon_max"], P["lat_min"], P["lat_max"]])
if use_zoom:
    zbox = gpd.GeoDataFrame(
        geometry=[box(min(P["lon_min"], P["lon_max"]),
                      min(P["lat_min"], P["lat_max"]),
                      max(P["lon_min"], P["lon_max"]),
                      max(P["lat_min"], P["lat_max"]))],
        crs=4326
    ).to_crs(utm_crs).total_bounds

# --- plot ---
fig, ax = plt.subplots(figsize=P["FIGSIZE"])
legend_items = []

if region_utm is not None and P["DRAW_REGION"]:
    region_utm.boundary.plot(ax=ax, color=P["REGION_COLOR"], linewidth=P["REGION_LW"],
                             alpha=P["REGION_ALPHA"], zorder=3)
    if P["SHOW_LEGEND"]:
        legend_items.append(Line2D([0],[0], color=P["REGION_COLOR"], lw=P["REGION_LW"],
                                   label=P["REGION_LABEL"]))

if edges_utm is not None and P["DRAW_EDGES"]:
    edges_utm.plot(ax=ax, color=P["EDGE_COLOR"], linewidth=P["EDGE_LW"],
                   alpha=P["EDGE_ALPHA"], zorder=1)
    if P["SHOW_LEGEND"]:
        legend_items.append(Line2D([0],[0], color=P["EDGE_COLOR"], lw=P["EDGE_LW"]*5,
                                   label=P["EDGE_LABEL"]))

if bldg_utm is not None and P["DRAW_BUILDINGS"]:
    bldg_utm.plot(ax=ax, color=P["BLDG_FACE"], edgecolor=P["BLDG_EDGE"],
                  linewidth=P["BLDG_LW"], alpha=P["BLDG_ALPHA"], zorder=2)
    if P["SHOW_LEGEND"]:
        legend_items.append(Patch(facecolor=P["BLDG_FACE"], edgecolor=P["BLDG_EDGE"],
                                  label=P["BLDG_LABEL"]))

if nodes_utm is not None and P["DRAW_NODES"]:
    nshow = nodes_utm if len(nodes_utm) <= P["NODE_MAX_PLOT"] else nodes_utm.sample(P["NODE_MAX_PLOT"], random_state=0)
    nshow.plot(ax=ax, color=P["NODE_COLOR"], markersize=P["NODE_SIZE"],
               alpha=P["NODE_ALPHA"], zorder=4)
    if P["SHOW_LEGEND"]:
        legend_items.append(Line2D([0],[0], marker="o", linestyle="None",
                                   markersize=max(4, int((P["NODE_SIZE"]**0.5)/1.5)),
                                   markerfacecolor=P["NODE_COLOR"], markeredgecolor="none",
                                   label=P["NODE_LABEL"]))

if use_zoom:
    xmin, ymin, xmax, ymax = zbox
    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

ax.set_aspect("equal")
ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
ax.set_title(P["TITLE"], fontsize=P["TITLE_FONTSIZE"])

if P["ADD_NORTH_ARROW"]:
    add_north(ax, x=P["NA_X"], y=P["NA_Y"], length=P["NA_LEN"],
              color=P["NA_COLOR"], lw=P["NA_LW"], fs=P["NA_FONTSIZE"], label=P["NA_LABEL"])

if P["ADD_SCALEBAR"]:
    add_scalebar(ax, dx=P["SB_DX"], units=P["SB_UNITS"], loc=P["SB_LOC"],
                 alpha=P["SB_BOX_ALPHA"], color=P["SB_COLOR"])

if P["SHOW_LEGEND"] and legend_items:
    ax.legend(handles=legend_items, loc=P["LEGEND_LOC"],
              frameon=P["LEGEND_FRAME"], facecolor=P["LEGEND_FACE"],
              edgecolor=P["LEGEND_EDGE"])

plt.show()

# --- save ---
out_img_dir = Path(r"./Outputs/Images"); out_img_dir.mkdir(parents=True, exist_ok=True)
out_png = out_img_dir / P["OUT_NAME"]
fig.savefig(out_png, dpi=P["DPI"], bbox_inches="tight")
print(f" Figure saved to: {out_png}")


In [ ]:
# =============================== A7) Summary ===============================

def _count(name):
    obj = globals().get(name, None)
    return 0 if (obj is None or getattr(obj, "empty", False)) else len(obj)

print("\n===== Section A Summary =====")
print(f"Buildings (downloaded RAW, pre-clip): {_count('bld_3857_raw')}")
print(f"Buildings (clipped to AOI):           {_count('bld_3857')}")
print(f"OSM Edges:                              {_count('edges')}")
print(f"OSM Nodes:                              {_count('nodes')}")
print("==========================================\n")


## B) Calculation of Damaged Buildings

In [ ]:
# =============================== B0) Imports ===============================
import os
from pathlib import Path

import numpy as np
import geopandas as gpd

import rasterio as rio
from rasterio.mask import mask
from rasterio.features import shapes
from rasterio.windows import from_bounds

from shapely.geometry import shape, box
from shapely.ops import unary_union
from shapely.prepared import prep

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from IPython.display import display, Image
import yaml
import time
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
# ============== B1) Data, Outputs, and User Parameters ===================

# Directories
DATA_DIR   = r"./Data"
OUT_BASE     = r"./Outputs"

# Fallback to Section A outputs if memory is missing
A_OUT_DIR    = os.path.join(OUT_BASE, "A")

# This section's (B) outputs
OUT_DIR      = os.path.join(OUT_BASE, "B")
IMAGES_DIR   = os.path.join(OUT_BASE, "Images")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

# Data
DPM_PATH = Path(os.path.join(DATA_DIR, require("DPM_FILE", str)))

# Outputs
EVAL_OUT_SHP      = Path(os.path.join(OUT_DIR, "Evaluated_Buildings.shp"))
DAMAGED_OUT_SHP   = Path(os.path.join(OUT_DIR, "Damaged_Buildings.shp"))
UNDAMAGED_OUT_SHP = Path(os.path.join(OUT_DIR, "Undamaged_Buildings.shp"))
DAMAGED_POLY_PATH = Path(os.path.join(OUT_DIR, "damaged_polygons_DPM.shp"))  # for plotting
DAMAGED_MASK_TIF  = Path(os.path.join(OUT_DIR, "AOI_DPM_DamagedMask_ge84.tif"))

# Threshold
DPM_THRESHOLD = require("DPM_THRESHOLD", float)


In [ ]:
# ==================== B2) Load AOI & Buildings =================

# AOI polygon (EPSG:4326) — prefer memory from Section A
if 'aoi_gdf_ll' not in globals():
    # Load from Section A outputs
    aoi_path = Path(os.path.join(A_OUT_DIR, "region_boundary.geojson"))
    if not aoi_path.exists():
        raise FileNotFoundError("AOI not in memory and region_boundary.geojson not found in Outputs_Final\\A.")
    aoi_gdf_ll = gpd.read_file(aoi_path).to_crs(4326)

# Buildings — prefer memory
if 'bld_3857' in globals() and bld_3857 is not None and not bld_3857.empty:
    buildings = bld_3857.copy()
else:
    buildings_path = Path(os.path.join(A_OUT_DIR, "buildings_selected_region.shp"))
    if not buildings_path.exists():
        raise FileNotFoundError("Buildings not in memory and buildings_selected_region.shp not found in Outputs_Final\\A.")
    buildings = gpd.read_file(buildings_path)
    if buildings.crs is None:
        raise ValueError("Loaded buildings have no CRS; please re-run Section A.")

# Clean geometries
buildings = buildings[~buildings.geometry.is_empty & buildings.geometry.notnull()].copy()


In [ ]:
# ========================= B3) Clip DPM raster to AOI =========================
with rio.open(DPM_PATH) as src:
    AOI_RCRS = aoi_gdf_ll.to_crs(src.crs)
    aoi_geom = [AOI_RCRS.union_all().__geo_interface__]
    out_img, out_transform = mask(src, aoi_geom, crop=True, all_touched=True)
    out_meta = src.meta.copy()
    out_meta.update({
        "height": out_img.shape[1],
        "width" : out_img.shape[2],
        "transform": out_transform,
        "driver": "GTiff",
    })
    nd   = src.nodata
    rcrs = src.crs

tif_clip = Path(os.path.join(OUT_DIR, "AOI_DPM_Clipped.tif"))
with rio.open(tif_clip, "w", **out_meta) as dst:
    dst.write(out_img)

print(f" Clipped DPM saved → {tif_clip}")


In [ ]:
# ========== B4) Threshold → damaged mask GeoTIFF → polygons =========
with rio.open(tif_clip) as src:
    arr = src.read(1)
    nd  = src.nodata
    t   = src.transform
    rcrs = src.crs

valid = (~np.isnan(arr)) if nd is None else (arr != nd)
damage_mask = (valid) & (arr >= DPM_THRESHOLD)
damage_mask_u8 = damage_mask.astype("uint8")     # 1 = damaged, 0 = not

# Save binary damaged mask
mask_meta = out_meta.copy()
mask_meta.update({"count": 1, "dtype": "uint8", "nodata": 0})
with rio.open(DAMAGED_MASK_TIF, "w", **mask_meta) as dst:
    dst.write(damage_mask_u8, 1)
print(f" Damaged binary mask saved → {DAMAGED_MASK_TIF}")

# Vectorize damaged pixels for plotting
dam_polys = [
    shape(geom)
    for geom, val in shapes(damage_mask_u8, mask=damage_mask_u8.astype(bool), transform=t)
    if val == 1
]
damaged_gdf = gpd.GeoDataFrame(geometry=dam_polys, crs=rcrs) if len(dam_polys) else gpd.GeoDataFrame(geometry=[], crs=rcrs)
damaged_gdf.to_file(DAMAGED_POLY_PATH)
print(f" Damaged polygons (for plotting) saved → {DAMAGED_POLY_PATH}")


In [ ]:
# =========== B5) Tiled Labeling ============

print("Initializing tiled labeling...")

# Tiling params
TILE_M    = require("TILE_M", float)
OVERLAP_M = require("OVERLAP_M", float)
DISSOLVE_IN_TILE = require("DISSOLVE_IN_TILE", bool)

# Read raster
print("Reading clipped raster...")
with rio.open(tif_clip) as ds:
    rcrs     = ds.crs
    r_bounds = ds.bounds
    nd       = ds.nodata

# Prepare buildings
print("Preparing building footprints...")
buildings_r = buildings.to_crs(rcrs) if buildings.crs != rcrs else buildings.copy()
buildings_r = buildings_r[~buildings_r.geometry.is_empty & buildings_r.geometry.notnull()].copy()

try:
    invalid_mask = ~buildings_r.is_valid
    if invalid_mask.any():
        print("Fixing invalid building geometries...")
        buildings_r.loc[invalid_mask, "geometry"] = buildings_r.loc[invalid_mask, "geometry"].buffer(0)
except Exception:
    print("Warning: Some building geometries could not be fixed.")

print(f"Total buildings: {len(buildings_r)}")

# Build tile grid
print("Building tile grid...")
minx, miny, maxx, maxy = r_bounds
xs = np.arange(minx, maxx, TILE_M - OVERLAP_M)
ys = np.arange(miny, maxy, TILE_M - OVERLAP_M)

tiles = []
for x in xs:
    x1 = min(x + TILE_M, maxx)
    if x1 <= x: continue
    for y in ys:
        y1 = min(y + TILE_M, maxy)
        if y1 <= y: continue
        tiles.append((x, y, x1, y1))

print(f"Number of tiles: {len(tiles)}")

damaged_mask_buildings = np.zeros(len(buildings_r), dtype=bool)
_ = buildings_r.sindex


def _vectorize_damaged_in_bounds(bounds):
    print("    Extracting damaged pixels...")
    with rio.open(tif_clip) as ds:
        win = from_bounds(*bounds, transform=ds.transform)
        win = win.intersection(rio.windows.Window(0, 0, ds.width, ds.height))

        if win.width <= 0 or win.height <= 0:
            print("    Tile outside raster bounds. Skipping.")
            return gpd.GeoDataFrame(geometry=[], crs=rcrs)

        arr = ds.read(1, window=win)
        sub_transform = ds.window_transform(win)

        valid = (~np.isnan(arr)) if ds.nodata is None else (arr != ds.nodata)
        dmg = (valid) & (arr >= DPM_THRESHOLD)

        if not dmg.any():
            print("    No damaged pixels found in tile.")
            return gpd.GeoDataFrame(geometry=[], crs=rcrs)

        print("    Vectorizing damaged regions...")
        dam_polys = [
            shape(geom) for geom, val in shapes(
                dmg.astype(np.uint8), mask=dmg, transform=sub_transform
            ) if val == 1
        ]

        if not dam_polys:
            print("    No valid polygons produced from damaged pixels.")
            return gpd.GeoDataFrame(geometry=[], crs=rcrs)

        if DISSOLVE_IN_TILE:
            print("    Dissolving polygons within tile...")
            u = unary_union(dam_polys)
            geoms = list(u.geoms) if getattr(u, "geom_type", "") == "MultiPolygon" else [u]
        else:
            geoms = dam_polys

        return gpd.GeoDataFrame(geometry=geoms, crs=rcrs)


# Main loop
for idx, tb in enumerate(tiles, start=1):

    print(f"\nProcessing tile {idx}/{len(tiles)}...")
    dmg_tile = _vectorize_damaged_in_bounds(tb)

    if dmg_tile.empty:
        print("    No damage polygons in this tile.")
        continue

    print("    Finding buildings that intersect tile...")
    try:
        L, R = buildings_r.sindex.query_bulk(
            gpd.GeoSeries([box(*tb)], crs=rcrs), predicate="intersects"
        )
        if R.size == 0:
            print("    No buildings found in this tile.")
            continue
        cand = buildings_r.iloc[np.unique(R)]
    except Exception:
        print("    Spatial index failed; using fallback method...")
        cand = buildings_r[buildings_r.intersects(box(*tb))]
        if cand.empty:
            print("    No buildings found in fallback method.")
            continue

    print(f"    Candidate buildings: {len(cand)}")
    print("    Checking which buildings intersect damaged areas...")

    p = prep(dmg_tile.unary_union)
    hit = cand.geometry.apply(p.intersects).to_numpy()

    hit_count = int(hit.sum())
    if hit_count > 0:
        print(f"    Buildings marked as damaged: {hit_count}")
        damaged_mask_buildings[cand.index.values[hit]] = True
    else:
        print("    No buildings damaged in this tile.")


# Final labeling
print("\nAssigning final damage labels...")

buildings_r["Damaged"] = np.where(damaged_mask_buildings, "yes", "no")
damaged_only   = buildings_r[buildings_r["Damaged"] == "yes"].copy()
undamaged_only = buildings_r[buildings_r["Damaged"] == "no"].copy()

print("Saving outputs...")
buildings_r.to_file(EVAL_OUT_SHP)
damaged_only.to_file(DAMAGED_OUT_SHP)
undamaged_only.to_file(UNDAMAGED_OUT_SHP)

print(f"Evaluated buildings: {len(buildings_r)}")
print(f"  Damaged   (≥ {DPM_THRESHOLD}): {len(damaged_only)}")
print(f"  Undamaged (< {DPM_THRESHOLD}): {len(undamaged_only)}")
print("Saved files:")
print(" ", EVAL_OUT_SHP)
print(" ", DAMAGED_OUT_SHP)
print(" ", UNDAMAGED_OUT_SHP)


In [ ]:
# =================== B6) PLOT: BUILDINGS + DAMAGED PIXELS ===================
P = {
    # ---- Output ----
    "OUT_NAME": "Buildings_with_DamagedPixels_LOCAL_UTM.png",  # output PNG filename

    # ---- Optional manual zoom in EPSG:4326 (set all to None for autoscale) ----
    "lon_min": 36.140,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.165,             # max longitude (None = auto)
    "lat_min": 36.190,             # min latitude  (None = auto)
    "lat_max": 36.205,             # max latitude  (None = auto)

    # ---- Figure & labels ----
    "FIGSIZE": (10, 9),            # figure size in inches (w, h)
    "DPI": 300,                    # export resolution
    "TITLE": "Buildings + Damaged Pixels (Local UTM, ≥ threshold)",
    "XLABEL": "Easting (m)",
    "YLABEL": "Northing (m)",

    # ---- Buildings style ----
    "BLDG_FACE": "#9e9e9e",        # fill color for all buildings
    "BLDG_EDGE": "none",           # outline color
    "BLDG_LW": 0.2,                # outline width (if not 'none')
    "BLDG_ALPHA": 1.0,             # layer transparency (0..1)

    # ---- Damaged polygons style ----
    "DMG_FACE": "red",             # fill color for damaged polygons
    "DMG_EDGE": "none",            # outline color
    "DMG_ALPHA": 0.5,              # layer transparency (0..1)

    # ---- AOI boundary (optional if available) ----
    "DRAW_AOI": True,              # draw AOI if aoi_gdf_ll exists
    "AOI_COLOR": "black",          # AOI boundary color
    "AOI_LW": 1.5,                 # AOI boundary width

    # ---- North arrow ----
    "ADD_NORTH_ARROW": True,       # draw north arrow
    "NA_X": 0.05,                  # arrow x-position (axes fraction)
    "NA_Y": 0.15,                  # arrow y-position (axes fraction)
    "NA_LEN": 0.08,                # arrow length  (axes fraction)
    "NA_LABEL": "N",               # arrow label
    "NA_COLOR": "black",           # arrow color
    "NA_LW": 2,                    # arrow line width
    "NA_FONTSIZE": 14,             # 'N' font size

    # ---- Scalebar ----
    "ADD_SCALEBAR": True,          # draw scalebar
    "SB_UNITS": "m",               # scalebar units
    "SB_LOC": "lower right",       # scalebar location
    "SB_BOX_ALPHA": 0.8,           # scalebar box transparency

    # ---- Legend ----
    "LEGEND_LOC": "upper right",   # legend location
    "LBL_BUILDINGS": "Buildings",  # legend label for buildings
    "LBL_DAMAGED": "Damaged pixels", # legend label for damaged polygons
    "LBL_AOI": "AOI Boundary"      # legend label for AOI boundary
}

# ----- Methodology -----

# Local UTM from damaged polygons
utm_crs = damaged_gdf.to_crs(4326).estimate_utm_crs()

# Reproject to local UTM
b_all_utm = buildings_r.to_crs(utm_crs)
dmg_utm   = damaged_gdf.to_crs(utm_crs)

# Optional lon/lat bbox → project to UTM and set frame
use_bbox = all(P[k] is not None for k in ["lon_min","lon_max","lat_min","lat_max"])
if use_bbox:
    aoi_ll  = gpd.GeoDataFrame(
        geometry=[box(min(P["lon_min"], P["lon_max"]),
                      min(P["lat_min"], P["lat_max"]),
                      max(P["lon_min"], P["lon_max"]),
                      max(P["lat_min"], P["lat_max"]))],
        crs=4326
    )
    xmin, ymin, xmax, ymax = aoi_ll.to_crs(utm_crs).total_bounds
else:
    bounds = [b_all_utm.total_bounds, dmg_utm.total_bounds]
    xmin = min(b[0] for b in bounds); ymin = min(b[1] for b in bounds)
    xmax = max(b[2] for b in bounds); ymax = max(b[3] for b in bounds)

# Plot
fig, ax = plt.subplots(figsize=P["FIGSIZE"])

b_all_utm.plot(ax=ax, color=P["BLDG_FACE"], edgecolor=P["BLDG_EDGE"],
               linewidth=P["BLDG_LW"], alpha=P["BLDG_ALPHA"], zorder=1)
dmg_utm.plot(ax=ax, facecolor=P["DMG_FACE"], edgecolor=P["DMG_EDGE"],
             alpha=P["DMG_ALPHA"], zorder=2)

# Optional AOI boundary
if P["DRAW_AOI"] and 'aoi_gdf_ll' in globals():
    aoi_gdf_ll.to_crs(utm_crs).boundary.plot(ax=ax, color=P["AOI_COLOR"],
                                             linewidth=P["AOI_LW"], zorder=5)

ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

# North arrow
if P["ADD_NORTH_ARROW"]:
    ax.annotate(P["NA_LABEL"], xy=(P["NA_X"], P["NA_Y"]), xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=P["NA_FONTSIZE"], fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                clip_on=False, zorder=20)

# Scalebar
if P["ADD_SCALEBAR"]:
    ax.add_artist(ScaleBar(1, P["SB_UNITS"], location=P["SB_LOC"], box_alpha=P["SB_BOX_ALPHA"]))

ax.set_aspect("equal"); ax.ticklabel_format(style="plain")
ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
ax.set_title(P["TITLE"])

# Legend
legend_elements = [
    Patch(facecolor=P["BLDG_FACE"], edgecolor="none", label=P["LBL_BUILDINGS"]),
    Patch(facecolor=P["DMG_FACE"], edgecolor="none", alpha=P["DMG_ALPHA"], label=P["LBL_DAMAGED"]),
]
if P["DRAW_AOI"] and 'aoi_gdf_ll' in globals():
    legend_elements.append(Line2D([0], [0], color=P["AOI_COLOR"], lw=P["AOI_LW"], label=P["LBL_AOI"]))
ax.legend(handles=legend_elements, loc=P["LEGEND_LOC"], frameon=True)

out_png = Path(IMAGES_DIR) / P["OUT_NAME"]
fig.savefig(out_png, dpi=P["DPI"], bbox_inches="tight")
plt.close(fig)
display(Image(filename=out_png))


In [ ]:
# ========== B7) PLOT: DAMAGED vs UNDAMAGED (LOCAL UTM) =========
P = {
    # ---- Output ----
    "OUT_NAME": "Damaged_vs_Undamaged_Buildings_LocalUTM.png",  # output PNG filename

    # ---- Manual zoom in EPSG:4326; set all to None for auto extent ----
    "lon_min": 36.900,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.930,             # max longitude (None = auto)
    "lat_min": 37.570,             # min latitude  (None = auto)
    "lat_max": 37.600,             # max latitude  (None = auto)

    # ---- Figure & labels ----
    "FIGSIZE": (10, 9),              # figure size (w, h) inches
    "DPI": 300,                      # export resolution
    "TITLE": "Buildings: Damaged (red) vs Undamaged (gray) — Local UTM",
    "XLABEL": "Easting (m)",
    "YLABEL": "Northing (m)",

    # ---- Undamaged style ----
    "UND_FACE": "#bdbdbd",           # fill color
    "UND_EDGE": "none",              # outline color
    "UND_LW": 0.2,                   # outline width
    "UND_ALPHA": 1.0,                # transparency (0..1)
    "LBL_UND": "Undamaged",          # legend label

    # ---- Damaged style ----
    "DAM_FACE": "#d73027",           # fill color
    "DAM_EDGE": "none",              # outline color
    "DAM_LW": 0.2,                   # outline width
    "DAM_ALPHA": 1.0,                # transparency (0..1)
    "LBL_DAM": "Damaged",            # legend label

    # ---- AOI boundary (optional) ----
    "DRAW_AOI": True,                # draw AOI boundary (expects aoi_gdf_ll upstream)
    "AOI_COLOR": "black",            # AOI boundary color
    "AOI_LW": 1.5,                   # AOI boundary width
    "LBL_AOI": "AOI Boundary",       # legend label

    # ---- North arrow ----
    "ADD_NORTH_ARROW": True,         # draw north arrow
    "NA_X": 0.05,                    # arrow x-position (axes fraction)
    "NA_Y": 0.15,                    # arrow y-position (axes fraction)
    "NA_LEN": 0.08,                  # arrow length (axes fraction)
    "NA_LABEL": "N",                 # arrow text
    "NA_COLOR": "black",             # arrow color
    "NA_LW": 2,                      # arrow line width
    "NA_FONTSIZE": 14,               # 'N' font size

    # ---- Scalebar ----
    "ADD_SCALEBAR": True,            # draw scalebar
    "SB_UNITS": "m",                 # scalebar units
    "SB_LOC": "lower right",         # scalebar location
    "SB_BOX_ALPHA": 0.8,             # scalebar box transparency

    # ---- Legend ----
    "LEGEND_LOC": "upper right"      # legend placement
}
# ---------------------------------------------------------------------------

# Choose UTM from labeled layers
seed = damaged_only if not damaged_only.empty else undamaged_only
utm_crs = seed.to_crs(4326).estimate_utm_crs()

# Reproject
dam_utm = damaged_only.to_crs(utm_crs)
und_utm = undamaged_only.to_crs(utm_crs)

# Optional lon/lat bbox → UTM frame
use_bbox = all(P[k] is not None for k in ("lon_min", "lon_max", "lat_min", "lat_max"))
if use_bbox:
    aoi_ll = gpd.GeoDataFrame(
        geometry=[box(min(P["lon_min"], P["lon_max"]),
                      min(P["lat_min"], P["lat_max"]),
                      max(P["lon_min"], P["lon_max"]),
                      max(P["lat_min"], P["lat_max"]))],
        crs=4326
    )
    xmin, ymin, xmax, ymax = aoi_ll.to_crs(utm_crs).total_bounds
else:
    bounds = [und_utm.total_bounds, dam_utm.total_bounds]
    xmin = min(b[0] for b in bounds); ymin = min(b[1] for b in bounds)
    xmax = max(b[2] for b in bounds); ymax = max(b[3] for b in bounds)

# Plot
fig, ax = plt.subplots(figsize=P["FIGSIZE"])

if not und_utm.empty:
    und_utm.plot(ax=ax, color=P["UND_FACE"], edgecolor=P["UND_EDGE"],
                 linewidth=P["UND_LW"], alpha=P["UND_ALPHA"], zorder=1)
if not dam_utm.empty:
    dam_utm.plot(ax=ax, color=P["DAM_FACE"], edgecolor=P["DAM_EDGE"],
                 linewidth=P["DAM_LW"], alpha=P["DAM_ALPHA"], zorder=2)

# Optional AOI boundary
if P["DRAW_AOI"] and 'aoi_gdf_ll' in globals():
    aoi_gdf_ll.to_crs(utm_crs).boundary.plot(ax=ax, color=P["AOI_COLOR"], linewidth=P["AOI_LW"], zorder=5)

ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

# North arrow
if P["ADD_NORTH_ARROW"]:
    ax.annotate(P["NA_LABEL"], xy=(P["NA_X"], P["NA_Y"]), xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=P["NA_FONTSIZE"], fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                clip_on=False, zorder=20)

# Scalebar
if P["ADD_SCALEBAR"]:
    ax.add_artist(ScaleBar(1, P["SB_UNITS"], location=P["SB_LOC"], box_alpha=P["SB_BOX_ALPHA"]))

ax.set_aspect("equal"); ax.ticklabel_format(style="plain")
ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
ax.set_title(P["TITLE"])

# Legend
legend_elements = [
    Patch(facecolor=P["UND_FACE"], edgecolor="none", label=P["LBL_UND"]),
    Patch(facecolor=P["DAM_FACE"], edgecolor="none", label=P["LBL_DAM"]),
]
if P["DRAW_AOI"] and 'aoi_gdf_ll' in globals():
    legend_elements.append(Line2D([0], [0], color=P["AOI_COLOR"], lw=P["AOI_LW"], label=P["LBL_AOI"]))
ax.legend(handles=legend_elements, loc=P["LEGEND_LOC"], frameon=True)

out_png = Path(IMAGES_DIR) / P["OUT_NAME"]
fig.savefig(out_png, dpi=P["DPI"], bbox_inches="tight")
plt.close(fig)
display(Image(filename=out_png))


In [ ]:
# ============================== B8) Summary ==============================
def _crs_str(crs):
    try:
        return str(crs) if crs is not None else "(no CRS)"
    except Exception:
        return "(unknown CRS)"

# Datasets
_b  = buildings_r
_bd = damaged_only
_bu = undamaged_only

# OSM nodes/edges
if 'nodes' in globals() and nodes is not None and not nodes.empty:
    _nodes = nodes
else:
    npath = Path(os.path.join(A_OUT_DIR, "osm_nodes.shp"))
    _nodes = gpd.read_file(npath) if npath.exists() else None

if 'edges' in globals() and edges is not None and not edges.empty:
    _edges = edges
else:
    epath = Path(os.path.join(A_OUT_DIR, "osm_edges.shp"))
    _edges = gpd.read_file(epath) if epath.exists() else None

# Damaged pixels
dam_pixels = 0
if DAMAGED_MASK_TIF.exists():
    with rio.open(DAMAGED_MASK_TIF) as src:
        arr = src.read(1)
        dam_pixels = int(np.count_nonzero(arr == 1))
        mask_crs = src.crs
else:
    mask_crs = None

print("===== Section B Summary (AOI only) =====")
print(f"Total buildings:           {len(_b):,}")
print(f"  Damaged (>= threshold):  {len(_bd):,}")
print(f"  Undamaged:               {len(_bu):,}")
print(f"Total OSM nodes:           {len(_nodes) if _nodes is not None else 0:,}")
print(f"Total OSM edges:           {len(_edges) if _edges is not None else 0:,}")
print(f"Total damaged pixels:      {dam_pixels:,}")

print("\n===== Output files (Section B) =====")
print("All Buildings (labeled):   ", EVAL_OUT_SHP)
print("Damaged Buildings:         ", DAMAGED_OUT_SHP)
print("Undamaged Buildings:       ", UNDAMAGED_OUT_SHP)
print("Damaged Pixels (mask):     ", DAMAGED_MASK_TIF)
print("Damaged Polygons:          ", DAMAGED_POLY_PATH)


## C) Affected Population Calculation

In [ ]:
# =============================== C0) Imports ===============================
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio as rio
from rasterio.mask import mask
from rasterio.features import shapes
from rasterio.windows import from_bounds

from shapely.geometry import shape, box

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from IPython.display import Image, display
import yaml
from matplotlib_scalebar.scalebar import ScaleBar


In [ ]:
# ============== C1) User Options, Data, Outputs =============

# Unified directories
DATA_DIR = r"./Data"
OUT_BASE   = r"./Outputs"

# Prior-section fallbacks
A_OUT_DIR  = os.path.join(OUT_BASE, "A")
B_OUT_DIR  = os.path.join(OUT_BASE, "B")

# This section's outputs
OUT_DIR    = os.path.join(OUT_BASE, "C")
IMAGES_DIR = os.path.join(OUT_BASE, "Images")
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

# Population raster
POP_RASTER = os.path.join(DATA_DIR, require("POP_FILE", str))

# Evaluated buildings from Section B
EVAL_OUT_SHP = (
    EVAL_OUT_SHP
    if "EVAL_OUT_SHP" in globals()
    else Path(os.path.join(B_OUT_DIR, "Evaluated_Buildings.shp"))
)

# Outputs for this section
OUT_SHP_VOL      = Path(os.path.join(OUT_DIR, "Evaluated_Buildings_withVol.shp"))
OUT_SHP_VOL_POP  = Path(os.path.join(OUT_DIR, "Evaluated_Buildings_withVol_Pop.shp"))

# Height settings
HEIGHT_COL     = "height"                  # keep as-is unless you prefer to read from config
DEFAULT_HEIGHT = optional("DEFAULT_HEIGHT")  # e.g., 3.0 to force a default height (meters)


In [ ]:
# ======== C2) Load AOI & Evaluated Buildings ========

# AOI polygon (EPSG:4326)
if "aoi_gdf_ll" not in globals() or aoi_gdf_ll is None or aoi_gdf_ll.empty:
    aoi_path = Path(os.path.join(A_OUT_DIR, "region_boundary.geojson"))
    if not aoi_path.exists():
        raise FileNotFoundError("AOI not in memory and region_boundary.geojson not found in Outputs_Final\\A.")
    aoi_gdf_ll = gpd.read_file(aoi_path).to_crs(4326)

# Evaluated buildings
if "buildings_r" in globals() and buildings_r is not None and not buildings_r.empty:
    b_eval = buildings_r.copy()
else:
    b_eval = gpd.read_file(EVAL_OUT_SHP)
    if b_eval.empty:
        raise RuntimeError("Evaluated_Buildings.shp is empty.")
    if b_eval.crs is None:
        raise ValueError("Evaluated_Buildings.shp has no CRS defined.")


In [ ]:
# ====================== C3) Compute Volume (m³) and Save =====================

# Ensure AOI strictness
try:
    b_eval = gpd.overlay(b_eval, aoi_gdf_ll.to_crs(b_eval.crs), how="intersection")
except Exception:
    b_eval = gpd.sjoin(b_eval, aoi_gdf_ll[["geometry"]].to_crs(b_eval.crs),
                       predicate="intersects", how="inner").drop(columns="index_right")

# Compute area in m² using a projected CRS (local UTM if needed)
if not getattr(b_eval.crs, "is_projected", False):
    utm_crs = b_eval.to_crs(4326).estimate_utm_crs()
    b_area = b_eval.to_crs(utm_crs)
else:
    b_area = b_eval

area_m2 = b_area.geometry.area.values

# Heights
if HEIGHT_COL in b_eval.columns:
    h = pd.to_numeric(b_eval[HEIGHT_COL], errors="coerce").fillna(0.0).to_numpy()
elif DEFAULT_HEIGHT is not None:
    h = np.full(len(b_eval), float(DEFAULT_HEIGHT), dtype=float)
else:
    raise ValueError(
        f"Height column '{HEIGHT_COL}' not found. "
        f"Either set HEIGHT_COL correctly or provide DEFAULT_HEIGHT."
    )

# Volume
b_eval["Volume"] = area_m2 * h

# Save
b_eval.to_file(OUT_SHP_VOL)
print(f" Added 'Volume' (m³) and saved → {OUT_SHP_VOL}")
print(f" Count: {len(b_eval):,} | CRS: {b_eval.crs}")


In [ ]:
# === C4) AOI Population → Allocate to Buildings by Volume (m³) ===

# Ensure Volume-enabled buildings
b = b_eval.copy() if "Volume" in b_eval.columns else gpd.read_file(OUT_SHP_VOL)
if b.empty:
    raise RuntimeError("Volume-enabled buildings are empty.")
if "Volume" not in b.columns:
    raise ValueError("Expected 'Volume' column in buildings.")

# Clip population raster to AOI and sum
with rio.open(POP_RASTER) as src:
    aoi_in_rcrs = aoi_gdf_ll.to_crs(src.crs)
    aoi_geom = [aoi_in_rcrs.union_all().__geo_interface__]
    pop_clip, _ = mask(src, aoi_geom, crop=True, all_touched=True)
    pop_arr = pop_clip[0].astype(float)
    nodata  = src.nodata
    if nodata is not None:
        pop_arr = np.where(pop_arr == nodata, 0.0, pop_arr)
    pop_arr = np.nan_to_num(pop_arr, nan=0.0)
    total_population = int(round(pop_arr.sum()))

print(f" Total population within AOI: {total_population:,}")

# Allocate ∝ Volume
vol = pd.to_numeric(b["Volume"], errors="coerce").to_numpy(dtype=float)
vol[~np.isfinite(vol)] = 0.0
vol_sum = float(vol.sum())
if vol_sum <= 0:
    raise ValueError("Total building Volume is zero/invalid; cannot allocate population.")

alloc   = (vol / vol_sum) * total_population
pop_int = np.floor(alloc).astype(int)
remainder = int(total_population - int(pop_int.sum()))
if remainder > 0:
    frac = alloc - pop_int
    top_idx = np.argsort(frac)[::-1][:remainder]
    pop_int[top_idx] += 1

b["Pop_Est"] = pop_int  # integer population per building
b.to_file(OUT_SHP_VOL_POP)
print(f" Saved buildings with Volume + Pop_Est → {OUT_SHP_VOL_POP}")
print(f"   Buildings: {len(b):,} | CRS: {b.crs}")
print(f"   Sum Pop_Est (should match AOI total): {int(b['Pop_Est'].sum()):,}")


In [ ]:
# ========== C5) Plot: Affected Population by Building ==========
# Preconditions
if b.empty:
    raise RuntimeError("No buildings to plot.")
if "Damaged" not in b.columns or "Pop_Est" not in b.columns:
    raise ValueError("Required columns 'Damaged' and 'Pop_Est' missing.")

# --------------------------- USER CONTROLS ---------------------------
P = {
    "OUT_NAME": "Affected_Population_by_Building.png",

    # Manual zoom in EPSG:4326 (set all to None for auto extent)
    "lon_min": 36.900,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.930,             # max longitude (None = auto)
    "lat_min": 37.570,             # min latitude  (None = auto)
    "lat_max": 37.600,             # max latitude  (None = auto)

    # Figure & labels
    "FIGSIZE": (10, 9),
    "DPI": 300,
    "TITLE": "Affected Population by Building",
    "XLABEL": "Easting (m)",
    "YLABEL": "Northing (m)",

    # Binning (0–10 … 100+) and palette (11 colors → 11 bins)
    "BIN_MIN": 0,
    "BIN_MAX": 100,
    "BIN_STEP": 10,
    "PALETTE": [
        "#fee5d9","#fcbba1","#fc9272","#fb6a4a",
        "#ef3b2c","#cb181d","#a50f15","#67000d",
        "#4a0010","#2e0006","#1a0004"
    ],

    # Styles
    "UND_FACE": "white", "UND_EDGE": "black", "UND_LW": 0.3, "UND_ALPHA": 1.0,
    "DAM_EDGE": "black", "DAM_LW": 0.2, "DAM_ALPHA": 1.0,

    # AOI boundary
    "DRAW_AOI": True, "AOI_COLOR": "black", "AOI_LW": 1.0,

    # North arrow & scalebar
    "ADD_NORTH_ARROW": True, "NA_X": 0.05, "NA_Y": 0.15, "NA_LEN": 0.08,
    "NA_LABEL": "N", "NA_COLOR": "black", "NA_LW": 2, "NA_FONTSIZE": 14,
    "ADD_SCALEBAR": True, "SB_UNITS": "m", "SB_LOC": "lower right",

    # Legend
    "LEGEND_LOC": "upper right"
}
# --------------------------------------------------------------------

# Reproject to local UTM for plotting in meters
utm_crs = b.to_crs(4326).estimate_utm_crs()
b_utm   = b.to_crs(utm_crs)
dam_utm = b_utm[b_utm["Damaged"] == "yes"].copy()
und_utm = b_utm[b_utm["Damaged"] == "no"].copy()

# Create equal-width bins up to BIN_MAX and a 100+ bin
edges      = np.arange(P["BIN_MIN"], P["BIN_MAX"] + P["BIN_STEP"], P["BIN_STEP"])
edges_ext  = np.append(edges, np.inf)
labels     = [f"{edges[i]}–{edges[i+1]}" for i in range(len(edges)-1)] + [f"{P['BIN_MAX']}+"]

dam_utm["pop_bin"] = pd.cut(
    dam_utm["Pop_Est"].astype(float),
    bins=edges_ext, right=False, labels=labels, include_lowest=True
)
palette = P["PALETTE"][:len(labels)]

# Optional zoom window (lon/lat → UTM)
use_bbox = all(P[k] is not None for k in ("lon_min", "lon_max", "lat_min", "lat_max"))
if use_bbox:
    zoom_ll = gpd.GeoDataFrame(
        geometry=[box(min(P["lon_min"], P["lon_max"]),
                      min(P["lat_min"], P["lat_max"]),
                      max(P["lon_min"], P["lon_max"]),
                      max(P["lat_min"], P["lat_max"]))],
        crs=4326
    )
    xmin, ymin, xmax, ymax = zoom_ll.to_crs(utm_crs).total_bounds
else:
    xmin, ymin, xmax, ymax = b_utm.total_bounds

# Plot
fig, ax = plt.subplots(figsize=P["FIGSIZE"])
legend_elems = []

# Undamaged outlines
if not und_utm.empty:
    und_utm.plot(ax=ax, facecolor=P["UND_FACE"], edgecolor=P["UND_EDGE"],
                 linewidth=P["UND_LW"], alpha=P["UND_ALPHA"], zorder=1)
    legend_elems.append(Patch(facecolor=P["UND_FACE"], edgecolor=P["UND_EDGE"], label="Undamaged"))

# Damaged buildings colored by pop_bin
for lab, color in zip(labels, palette):
    sub = dam_utm[dam_utm["pop_bin"] == lab]
    if not sub.empty:
        sub.plot(ax=ax, facecolor=color, edgecolor=P["DAM_EDGE"],
                 linewidth=P["DAM_LW"], alpha=P["DAM_ALPHA"], zorder=2)
        legend_elems.append(Patch(facecolor=color, edgecolor=P["DAM_EDGE"], label=f"Affected Pop. {lab}"))

# AOI boundary
if P["DRAW_AOI"] and 'aoi_gdf_ll' in globals():
    aoi_gdf_ll.to_crs(utm_crs).boundary.plot(ax=ax, color=P["AOI_COLOR"], linewidth=P["AOI_LW"], zorder=4)
    legend_elems.append(Line2D([0], [0], color=P["AOI_COLOR"], lw=P["AOI_LW"], label="AOI Boundary"))

# Frame & decor
ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

if P["ADD_NORTH_ARROW"]:
    ax.annotate(P["NA_LABEL"], xy=(P["NA_X"], P["NA_Y"]),
                xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=P["NA_FONTSIZE"], fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                clip_on=False, zorder=20)

if P["ADD_SCALEBAR"]:
    ax.add_artist(ScaleBar(1, P["SB_UNITS"], location=P["SB_LOC"]))

ax.set_aspect("equal"); ax.ticklabel_format(style="plain")
ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
ax.set_title(P["TITLE"])
ax.legend(handles=legend_elems, loc=P["LEGEND_LOC"], frameon=True)

out_png = Path(IMAGES_DIR) / P["OUT_NAME"]
fig.savefig(out_png, dpi=P["DPI"], bbox_inches="tight")
plt.show()
print(f" Saved: {out_png}")


In [ ]:
# =============================== C6) Summary ===============================
if "Damaged" in b.columns and "Pop_Est" in b.columns:
    affected_pop   = int(b.loc[b["Damaged"] == "yes", "Pop_Est"].sum())
    unaffected_pop = int(b.loc[b["Damaged"] == "no",  "Pop_Est"].sum())
    total_pop      = affected_pop + unaffected_pop

    print("\n Population Summary (within AOI):")
    print(f"   Affected population (Damaged buildings):     {affected_pop:,}")
    print(f"   Unaffected population (Undamaged buildings): {unaffected_pop:,}")
    print(f"   Total population (check):                    {total_pop:,}")
else:
    print(" Columns 'Damaged' and/or 'Pop_Est' not found in building dataset.")


## D) Relief Calculation

In [ ]:
# =============================== D0) Imports ===============================
import os
import math
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

import geopandas as gpd
import yaml


In [ ]:
# ============== D1) Data, Outputs, and Constants  =============
OUT_BASE    = r"./Outputs"
C_OUT_DIR   = os.path.join(OUT_BASE, "C")
IMAGES_DIR  = os.path.join(OUT_BASE, "Images")
os.makedirs(IMAGES_DIR, exist_ok=True)

# Fallback shapefile from Section C (buildings with Volume + Pop_Est + Damaged)
C_BLD_POP_SHP = os.path.join(C_OUT_DIR, "Evaluated_Buildings_withVol_Pop.shp")

# WHO / Sphere-like planning constants (with 10% buffer)
GUIDELINES = require("GUIDELINES", dict)

# Planning durations
DUR_AFFECTED   = require("DUR_AFFECTED", int)   # days
DUR_UNAFFECTED = require("DUR_UNAFFECTED", int) # days


In [ ]:
# =================== D2) Get population by damage class ======================
# Totals for Damaged==yes and Damaged==no.

def _load_buildings_memory_or_c():
    if "b" in globals() and b is not None and not b.empty:
        return b
    if os.path.exists(C_BLD_POP_SHP):
        return gpd.read_file(C_BLD_POP_SHP)
    raise FileNotFoundError("No buildings with Pop_Est available in memory or at Outputs_Final\\C.")

b_src = _load_buildings_memory_or_c()
if "Damaged" not in b_src.columns or "Pop_Est" not in b_src.columns:
    raise ValueError("Buildings data must include 'Damaged' and 'Pop_Est' columns.")

# Normalize labels and compute class totals
b_src["Damaged"] = b_src["Damaged"].astype(str).str.strip().str.lower()
affected_pop   = int(pd.to_numeric(b_src.loc[b_src["Damaged"] == "yes", "Pop_Est"], errors="coerce").fillna(0).sum())
unaffected_pop = int(pd.to_numeric(b_src.loc[b_src["Damaged"] == "no",  "Pop_Est"], errors="coerce").fillna(0).sum())
total_pop      = affected_pop + unaffected_pop

print(f" Population — Affected (90d): {affected_pop:,} | Unaffected (14d): {unaffected_pop:,} | Total: {total_pop:,}")


In [ ]:
# ========================== D3) Supply calculators ===========================
def calc_water_liters(pop, days):
    return pop * GUIDELINES["water_l_per_person_per_day"] * days * GUIDELINES["buffer_factor"]

def calc_food_kcal(pop, days):
    return pop * GUIDELINES["food_kcal_per_person_per_day"] * days * GUIDELINES["buffer_factor"]

def calc_shelter_area(pop):
    # Shelter capacity buffer not applied
    return pop * GUIDELINES["shelter_m2_per_person"]

def calc_iehk_kits(pop, days):
    if pop <= 0:
        return 0
    kits_fraction = (pop / GUIDELINES["iehk_pop_served"]) * (days / GUIDELINES["iehk_duration_days"])
    return math.ceil(kits_fraction)


In [ ]:
# =================== D4) Compute Results for both conditions =====================
records = []

# Affected (90d)
aw_liters  = calc_water_liters(affected_pop, DUR_AFFECTED)
af_kcal    = calc_food_kcal(affected_pop, DUR_AFFECTED)
as_m2      = calc_shelter_area(affected_pop)
ai_kits    = calc_iehk_kits(affected_pop, DUR_AFFECTED)

records.append({
    "Class": "Affected (90d)", "Days": DUR_AFFECTED, "Population": affected_pop,
    "Water_L": aw_liters, "Water_m3": aw_liters/1000.0,
    "Food_kcal": af_kcal, "Food_Mkcal": af_kcal/1e6,
    "Shelter_m2": as_m2, "IEHK_kits": ai_kits
})

# Unaffected (14d)
uw_liters  = calc_water_liters(unaffected_pop, DUR_UNAFFECTED)
uf_kcal    = calc_food_kcal(unaffected_pop, DUR_UNAFFECTED)
us_m2      = calc_shelter_area(unaffected_pop)
ui_kits    = calc_iehk_kits(unaffected_pop, DUR_UNAFFECTED)

records.append({
    "Class": "Unaffected (14d)", "Days": DUR_UNAFFECTED, "Population": unaffected_pop,
    "Water_L": uw_liters, "Water_m3": uw_liters/1000.0,
    "Food_kcal": uf_kcal, "Food_Mkcal": uf_kcal/1e6,
    "Shelter_m2": us_m2, "IEHK_kits": ui_kits
})

df_supply = pd.DataFrame.from_records(records)

# Totals for final summary
totals = {
    "Population_total": total_pop,
    "Water_L_total":   df_supply["Water_L"].sum(),
    "Water_m3_total":  df_supply["Water_m3"].sum(),
    "Food_kcal_total": df_supply["Food_kcal"].sum(),
    "Food_Mkcal_total":df_supply["Food_Mkcal"].sum(),
    "Shelter_m2_total":df_supply["Shelter_m2"].sum(),
    "IEHK_kits_total": int(df_supply["IEHK_kits"].sum()),  # already ceil per class
}


In [ ]:
# ========================= D5) All Bar Charts ===============================
def _series_with_total(df, colname):
    vals = {
        "Affected (90d)":   float(df.loc[df["Class"] == "Affected (90d)", colname].iloc[0]) if (df["Class"] == "Affected (90d)").any() else 0.0,
        "Unaffected (14d)": float(df.loc[df["Class"] == "Unaffected (14d)", colname].iloc[0]) if (df["Class"] == "Unaffected (14d)").any() else 0.0,
    }
    vals["Total"] = vals["Affected (90d)"] + vals["Unaffected (14d)"]
    labels = ["Affected (90d)", "Unaffected (14d)", "Total"]
    values = [vals[l] for l in labels]
    return labels, values


def _bar_and_save(labels, values, title, ylabel, fname, value_fmt="{:,.0f}"):
    COLORS = {
        "Affected (90d)": "#d73027",   # red
        "Unaffected (14d)": "#9e9e9e", # gray
        "Damaged": "#d73027",
        "Undamaged": "#9e9e9e",
        "Total": "#2e2e2e",            # dark
    }
    bar_colors = [COLORS.get(l, "#9e9e9e") for l in labels]

    plt.figure(figsize=(6, 4))
    bars = plt.bar(labels, values, edgecolor="black", color=bar_colors)
    for bar in bars:
        h = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, h, value_fmt.format(h),
                 ha="center", va="bottom", fontsize=9)
    plt.title(title, fontsize=12)
    plt.ylabel(ylabel)
    plt.tight_layout()

    out_png = Path(IMAGES_DIR) / fname
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close()

    try:
        display(Image(filename=str(out_png), width=300))
    except Exception:
        pass
    print(f" Saved: {out_png}")

#  1) Water (m³)
_labels, _vals = _series_with_total(df_supply, "Water_m3")
_bar_and_save(_labels, _vals, "Water Required (m³)", "Total Water (m³)",
              "D_Water_required_m3_from_memory.png", "{:,.0f}")

#  2) Food (Million kcal)
_labels, _vals = _series_with_total(df_supply, "Food_Mkcal")
_bar_and_save(_labels, _vals, "Food Required (Million kcal)", "Total Food (Million kcal)",
              "D_Food_required_MillionKcal_from_memory.png", "{:,.0f}")

#  3) Shelter (m²)
_labels, _vals = _series_with_total(df_supply, "Shelter_m2")
_bar_and_save(_labels, _vals, "Shelter Area Needed (m²)", "Total Shelter (m²)",
              "D_Shelter_area_required_m2_from_memory.png", "{:,.0f}")

#  4) IEHK kits
_labels, _vals = _series_with_total(df_supply, "IEHK_kits")
_bar_and_save(_labels, _vals, "Medical IEHK Kits Required", "Equivalent IEHK Kits",
              "D_IEHK_kits_required_from_memory.png", "{:,.0f}")

#  5) Buildings: Damaged vs Undamaged + Total
_b = b_src if ("b_src" in globals() and b_src is not None) else _load_buildings_memory_or_c()
_dam = int((_b["Damaged"].astype(str).str.lower() == "yes").sum())
_und = int((_b["Damaged"].astype(str).str.lower() == "no").sum())
labels_bld = ["Damaged", "Undamaged", "Total"]
vals_bld   = [_dam, _und, _dam + _und]

_bar_and_save(labels_bld, vals_bld,
              "Buildings: Damaged vs Undamaged",
              "Number of Buildings",
              "D_Buildings_Damaged_vs_Undamaged.png", "{:,.0f}")

#  6) Population: Affected vs Unaffected + Total
labels_pop = ["Affected (90d)", "Unaffected (14d)", "Total"]
vals_pop   = [affected_pop, unaffected_pop, total_pop]

_bar_and_save(labels_pop, vals_pop,
              "Population: Affected vs Unaffected",
              "People",
              "D_Population_Affected_vs_Unaffected.png", "{:,.0f}")


In [ ]:
# ========================= D6) Summary =========================
def _fmt(x, unit=""):
    if isinstance(x, (int, float)):
        return f"{x:,.0f}{unit}"
    return str(x)

print("\n===== HUMANITARIAN SUPPLY SUMMARY  =====")

# Prepare list for export table
summary_rows = []

# Per horizon (Affected / Unaffected)
for _, row in df_supply.iterrows():
    cls  = row["Class"]
    days = row["Days"]
    pop  = int(row["Population"])
    summary_rows.append({
        "Category": cls,
        "People": pop,
        "Duration_days": days,
        "Water_L": round(row["Water_L"], 2),
        "Water_m3": round(row["Water_m3"], 2),
        "Food_kcal": round(row["Food_kcal"], 2),
        "Food_Mkcal": round(row["Food_Mkcal"], 2),
        "Shelter_m2": round(row["Shelter_m2"], 2),
        "IEHK_kits": int(row["IEHK_kits"]),
    })

    print(f"\n— {cls} —")
    print(f"  People:            {_fmt(pop)}")
    print(f"  Duration:          {days} days")
    print(f"  Water:             {_fmt(row['Water_L'], ' L')}  ({_fmt(row['Water_m3'])} m³)")
    print(f"  Food:              {_fmt(row['Food_kcal'])} kcal  ({_fmt(row['Food_Mkcal'])} million kcal)")
    print(f"  Shelter:           {_fmt(row['Shelter_m2'])} m² (min {_fmt(GUIDELINES['shelter_m2_per_person'])} m²/person)")
    print(f"  IEHK Kits (ceil):  {_fmt(row['IEHK_kits'])}")

# Totals
summary_rows.append({
    "Category": "TOTAL",
    "People": int(totals["Population_total"]),
    "Duration_days": "Mixed (90+14)",
    "Water_L": round(totals["Water_L_total"], 2),
    "Water_m3": round(totals["Water_m3_total"], 2),
    "Food_kcal": round(totals["Food_kcal_total"], 2),
    "Food_Mkcal": round(totals["Food_Mkcal_total"], 2),
    "Shelter_m2": round(totals["Shelter_m2_total"], 2),
    "IEHK_kits": int(totals["IEHK_kits_total"]),
})

print("\n— TOTALS (Affected 90d + Unaffected 14d) —")
print(f"  Total People:      {_fmt(totals['Population_total'])}")
print(f"  Total Water:       {_fmt(totals['Water_L_total'], ' L')}  ({_fmt(totals['Water_m3_total'])} m³)")
print(f"  Total Food:        {_fmt(totals['Food_kcal_total'])} kcal  ({_fmt(totals['Food_Mkcal_total'])} million kcal)")
print(f"  Total Shelter:     {_fmt(totals['Shelter_m2_total'])} m²")
print(f"  Total IEHK Kits:   {_fmt(totals['IEHK_kits_total'])}")

print("\nAssumptions: 20 L p⁻¹ d⁻¹ water, 2,100 kcal p⁻¹ d⁻¹ food, 4 m² p⁻¹ shelter; "
      "IEHK = 10 000 people per 90 days; 10 % buffer applied to water & food.")
print("Durations: Affected = 90 days, Unaffected = 14 days.")
print("================================================================")

# --- Save summary to CSV (under Outputs\D) ---
D_OUT_DIR = Path(OUT_BASE) / "D"
D_OUT_DIR.mkdir(parents=True, exist_ok=True)
out_summary_csv = D_OUT_DIR / "D_Supply_Summary_Table.csv"

df_summary = pd.DataFrame(summary_rows)
df_summary.to_csv(out_summary_csv, index=False)

print(f" Summary table saved → {out_summary_csv}")
display(df_summary)


## E) Buffer and Damaged Edge Calculation

In [ ]:
# =============================== E0) Imports ===============================
import os
import numpy as np
import geopandas as gpd
from pathlib import Path

from shapely.geometry import box
from shapely.ops import unary_union
from shapely.prepared import prep

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from IPython.display import Image, display
import yaml


In [ ]:
# =============================== E1) User Options & Paths ===============================
OUT_BASE   = r"./Outputs"
A_OUT_DIR  = os.path.join(OUT_BASE, "A")
B_OUT_DIR  = os.path.join(OUT_BASE, "B")
C_OUT_DIR  = os.path.join(OUT_BASE, "C")
E_OUT_DIR  = os.path.join(OUT_BASE, "E")
IMAGES_DIR = os.path.join(OUT_BASE, "Images")
os.makedirs(E_OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

# --- User-controllable numerical parameters from Configuration ---
HEIGHT_COL        = require("HEIGHT_COL", str)      # building height column
BUFFER_FALLBACK_M = require("BUFFER_FALLBACK_M", float)  # meters when height missing
SIMPLIFY_TOL_M    = require("SIMPLIFY_TOL_M", float)     # simplify buffers for speed
TILE_M            = require("TILE_M_E", float)      # tile side in meters (E section)
OVERLAP_M         = require("OVERLAP_M_E", float)   # overlap between tiles

# --- Data: damaged buildings ---
if 'damaged_only' in globals() and damaged_only is not None and not damaged_only.empty:
    bldg_damaged_src = damaged_only.copy()
elif 'b' in globals() and b is not None and not b.empty and ("Damaged" in b.columns):
    bldg_damaged_src = b[b["Damaged"].astype(str).str.lower().eq("yes")].copy()
elif 'buildings_r' in globals() and buildings_r is not None and not buildings_r.empty and ("Damaged" in buildings_r.columns):
    bldg_damaged_src = buildings_r[buildings_r["Damaged"].astype(str).str.lower().eq("yes")].copy()
else:
    c_eval_pop = os.path.join(C_OUT_DIR, "Evaluated_Buildings_withVol_Pop.shp")
    if os.path.exists(c_eval_pop):
        _tmp = gpd.read_file(c_eval_pop)
        if "Damaged" not in _tmp.columns:
            raise RuntimeError("Fallback buildings file lacks 'Damaged' column.")
        bldg_damaged_src = _tmp[_tmp["Damaged"].astype(str).str.lower().eq("yes")].copy()
    else:
        raise RuntimeError("No damaged buildings found (memory/C outputs unavailable).")

if bldg_damaged_src.empty:
    raise SystemExit("No damaged buildings; nothing to buffer.")

# --- Data: OSM edges ---
if 'edges' in globals() and isinstance(edges, gpd.GeoDataFrame) and not edges.empty:
    edges_src = edges.copy()
else:
    edges_path = os.path.join(A_OUT_DIR, "osm_edges.shp")
    if not os.path.exists(edges_path):
        raise RuntimeError("OSM edges not found in memory or Outputs_Final\\A.")
    edges_src = gpd.read_file(edges_path)
    if edges_src is None or edges_src.empty:
        raise RuntimeError("Loaded edges are empty.")

# --- Output file paths ---
EVAL_EDGES_SHP        = Path(os.path.join(E_OUT_DIR, "Evaluated_Edges.shp"))
DAMAGED_EDGES_SHP     = Path(os.path.join(E_OUT_DIR, "Damaged_Edges.shp"))
UNDAMAGED_EDGES_SHP   = Path(os.path.join(E_OUT_DIR, "Undamaged_Edges.shp"))
BUFFER_ISLANDS_GPKG   = Path(os.path.join(E_OUT_DIR, "Buffer_Damaged_Buildings.gpkg"))
PER_BUILDING_BUFF_GPKG= Path(os.path.join(E_OUT_DIR, "PerBuilding_Buffers.gpkg"))


In [ ]:
# ======== E2) Local UTM =========
seed_wgs84 = bldg_damaged_src.to_crs(4326)
utm_crs = seed_wgs84.estimate_utm_crs()
print("Local UTM for buffers & edge tests →", utm_crs)

bldg_damaged = bldg_damaged_src.to_crs(utm_crs)
edges_utm    = edges_src.to_crs(utm_crs)

# Cheap validity repairs
try:
    inv_b = ~bldg_damaged.is_valid
    if inv_b.any():
        bldg_damaged.loc[inv_b, "geometry"] = bldg_damaged.loc[inv_b, "geometry"].buffer(0)
    inv_e = ~edges_utm.is_valid
    if inv_e.any():
        edges_utm.loc[inv_e, "geometry"] = edges_utm.loc[inv_e, "geometry"].buffer(0)
except Exception:
    pass


In [ ]:
# ============== E3) Per-building buffers  ==============

if HEIGHT_COL in bldg_damaged.columns:
    h_m = gpd.pd.to_numeric(bldg_damaged[HEIGHT_COL], errors='coerce')
else:
    h_m = gpd.pd.Series(np.nan, index=bldg_damaged.index, dtype=float)

h_m = h_m.fillna(BUFFER_FALLBACK_M).astype(float)
bldg_damaged["buf_m"] = h_m.values
print(f"Buffer stats (m): min={h_m.min():.2f}, median={h_m.median():.2f}, max={h_m.max():.2f}, n={len(h_m)}")

# Geometry-by-geometry buffers
per_buffers = [geom.buffer(dist) for geom, dist in zip(bldg_damaged.geometry, bldg_damaged["buf_m"])]
per_buffer_gdf = gpd.GeoDataFrame(
    bldg_damaged.drop(columns="geometry").copy(),
    geometry=per_buffers,
    crs=utm_crs
)
print(f"Per-building buffers created: {len(per_buffer_gdf)}")


In [ ]:
# ============== E4) Dissolve buffers → islands ==============
dissolved = unary_union(per_buffer_gdf.geometry.values)
islands = list(dissolved.geoms) if getattr(dissolved, "geom_type", "") == "MultiPolygon" else [dissolved]

buffer_islands = gpd.GeoDataFrame(
    {"island_id": range(len(islands))},
    geometry=islands,
    crs=utm_crs
)
print(f"Dissolved islands: {len(buffer_islands)}")

# Optional light simplify for speed
buffers_simpl = buffer_islands.copy()
buffers_simpl["geometry"] = buffers_simpl.geometry.simplify(SIMPLIFY_TOL_M, preserve_topology=True)

# Save QA layers
if not buffer_islands.empty: buffer_islands.to_file(BUFFER_ISLANDS_GPKG, driver="GPKG")
if not per_buffer_gdf.empty: per_buffer_gdf.to_file(PER_BUILDING_BUFF_GPKG, driver="GPKG")


In [ ]:
# ===================== E5) Tile the AOI and classify edges ====================
if buffers_simpl.empty:
    raise SystemExit("No buffer islands exist; cannot evaluate edges.")

minx, miny, maxx, maxy = buffers_simpl.total_bounds
xs = np.arange(minx, maxx, TILE_M - OVERLAP_M)
ys = np.arange(miny, maxy, TILE_M - OVERLAP_M)

# Pre-build spatial index
_ = edges_utm.sindex
edge_damaged_mask = np.zeros(len(edges_utm), dtype=bool)

for x in xs:
    x1 = min(x + TILE_M, maxx)
    if x1 <= x: continue
    for y in ys:
        y1 = min(y + TILE_M, maxy)
        if y1 <= y: continue

        tile_box_geom = box(x, y, x1, y1)

        # Candidate islands
        try:
            Lb, Rb = buffers_simpl.sindex.query_bulk(
                gpd.GeoSeries([tile_box_geom], crs=utm_crs), predicate="intersects"
            )
            if Rb.size == 0: 
                continue
            isl_tile = buffers_simpl.iloc[np.unique(Rb)]
        except Exception:
            isl_tile = buffers_simpl[buffers_simpl.intersects(tile_box_geom)]
            if isl_tile.empty: 
                continue

        p = prep(isl_tile.union_all())

        # Candidate edges
        try:
            Le, Re = edges_utm.sindex.query_bulk(
                gpd.GeoSeries([tile_box_geom], crs=utm_crs), predicate="intersects"
            )
            if Re.size == 0:
                continue
            cand = edges_utm.iloc[np.unique(Re)]
        except Exception:
            cand = edges_utm[edges_utm.intersects(tile_box_geom)]
            if cand.empty: 
                continue

        # Any intersection → damaged
        hit = cand.geometry.apply(p.intersects).to_numpy()
        if hit.any():
            edge_damaged_mask[cand.index.values[hit]] = True


In [ ]:
# ============== E6) Build evaluated / damaged / undamaged edge layers ==============
edges_eval = edges_utm.copy()
edges_eval["Damaged"] = np.where(edge_damaged_mask, "yes", "no")

edges_dmg = edges_eval.loc[edges_eval["Damaged"] == "yes"].copy()
edges_ok  = edges_eval.loc[edges_eval["Damaged"] == "no"].copy()

# Keep geometry-only in the split outputs
keep_cols = []
if keep_cols:
    keep_cols = [c for c in keep_cols if c in edges_eval.columns]
    edges_dmg = gpd.GeoDataFrame(edges_dmg[keep_cols].copy(), geometry=edges_dmg.geometry, crs=edges_dmg.crs)
    edges_ok  = gpd.GeoDataFrame(edges_ok[keep_cols].copy(),  geometry=edges_ok.geometry,  crs=edges_ok.crs)
else:
    edges_dmg = gpd.GeoDataFrame(geometry=edges_dmg.geometry, crs=edges_dmg.crs)
    edges_ok  = gpd.GeoDataFrame(geometry=edges_ok.geometry,  crs=edges_ok.crs)


In [ ]:
# =============================== E7) Save edges ===============================
edges_eval.to_file(EVAL_EDGES_SHP)
edges_dmg.to_file(DAMAGED_EDGES_SHP)
edges_ok.to_file(UNDAMAGED_EDGES_SHP)

print(f" Evaluated edges: {len(edges_eval)}")
print(f"  Damaged   (touches buffer): {len(edges_dmg)}")
print(f"  Undamaged:                  {len(edges_ok)}")
print(" Saved:")
print("  ", EVAL_EDGES_SHP)
print("  ", DAMAGED_EDGES_SHP)
print("  ", UNDAMAGED_EDGES_SHP)


In [ ]:
# ============ E8) Plot: Buffers, Buildings, Damaged (Local UTM) ============
P = {
    # ---- Output ----
    "OUT_NAME": "Buffer_Zones.png",        # output PNG filename (saved under IMAGES_DIR)

    # ---- Manual zoom in EPSG:4326 (set all to None for auto extent) ----
    "lon_min": 36.900,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.930,             # max longitude (None = auto)
    "lat_min": 37.570,             # min latitude  (None = auto)
    "lat_max": 37.600,             # max latitude  (None = auto)

    # ---- Figure & labels ----
    "FIGSIZE": (10, 9),                    # figure size (w, h) inches
    "DPI": 300,                            # export resolution
    "TITLE": "Buffer Zones, Buildings, and Damaged Buildings",
    "XLABEL": "Easting (m)",               # x-axis label (projected meters)
    "YLABEL": "Northing (m)",              # y-axis label (projected meters)

    # ---- Buffer polygons style ----
    "BUF_FACE": "black",                   # buffer fill color
    "BUF_EDGE": "none",                    # buffer outline color
    "BUF_ALPHA": 0.5,                      # buffer transparency

    # ---- All buildings (context) ----
    "BLDG_FACE": "white",                  # building fill color
    "BLDG_EDGE": "black",                  # building outline color
    "BLDG_LW": 0.3,                        # building outline width
    "BLDG_ALPHA": 1.0,                     # building transparency

    # ---- Damaged buildings ----
    "DAM_FACE": "#d73027",                 # damaged fill color
    "DAM_EDGE": "black",                   # damaged outline color
    "DAM_LW": 0.2,                         # damaged outline width
    "DAM_ALPHA": 1.0,                      # damaged transparency

    # ---- AOI boundary (optional) ----
    "DRAW_AOI": True,                      # draw AOI boundary (expects aoi_gdf_ll upstream)
    "AOI_COLOR": "black",                  # AOI line color
    "AOI_LW": 1.0,                         # AOI line width
    "LBL_AOI": "AOI Boundary",             # legend label

    # ---- North arrow & scalebar ----
    "ADD_NORTH_ARROW": True,               # draw north arrow
    "NA_X": 0.05, "NA_Y": 0.15,            # arrow position (axes fraction)
    "NA_LEN": 0.08,                        # arrow length (axes fraction)
    "NA_LABEL": "N",                       # arrow text
    "NA_COLOR": "black",                   # arrow color
    "NA_LW": 2,                            # arrow line width
    "NA_FONTSIZE": 14,                     # 'N' font size
    "ADD_SCALEBAR": True,                  # draw scalebar
    "SB_UNITS": "m",                       # scalebar units
    "SB_LOC": "lower right",               # scalebar location

    # ---- Legend ----
    "LBL_BUFFERS": "Buffer zones",
    "LBL_BUILDINGS": "Buildings",
    "LBL_DAMAGED": "Damaged buildings",
    "LEGEND_LOC": "upper right"            # legend placement
}
# ------------------------------------------------------------------------------

# Buildings "Damaged"
if 'b' in globals() and isinstance(b, gpd.GeoDataFrame) and not b.empty and ("Damaged" in b.columns):
    b_src = b.copy()
elif 'buildings_r' in globals() and isinstance(buildings_r, gpd.GeoDataFrame) and not buildings_r.empty and ("Damaged" in buildings_r.columns):
    b_src = buildings_r.copy()
else:
    eval_b = os.path.join(B_OUT_DIR, "Evaluated_Buildings.shp")
    if not os.path.exists(eval_b):
        raise RuntimeError("No evaluated buildings found for plotting.")
    b_src = gpd.read_file(eval_b)
    if "Damaged" not in b_src.columns:
        raise ValueError("Loaded buildings file has no 'Damaged' column.")

# Buffers fallback to per-building
buffers_src = buffer_islands if 'buffer_islands' in globals() else per_buffer_gdf
if buffers_src is None or buffers_src.empty:
    raise RuntimeError("No buffer polygons found for plotting.")

# Optional AOI
aoi_src = aoi_gdf_ll if 'aoi_gdf_ll' in globals() else None

# Reproject to local UTM
seed = buffers_src if not buffers_src.empty else b_src
utm_plot    = seed.to_crs(4326).estimate_utm_crs()
b_utm       = b_src.to_crs(utm_plot)
buffers_utm = buffers_src.to_crs(utm_plot)
aoi_utm     = aoi_src.to_crs(utm_plot) if isinstance(aoi_src, gpd.GeoDataFrame) else None
dam_utm     = b_utm[b_utm["Damaged"].astype(str).str.lower().eq("yes")].copy()

# Frame
use_bbox = all(P[k] is not None for k in ("lon_min","lon_max","lat_min","lat_max"))
if use_bbox:
    aoi_ll_zoom = gpd.GeoDataFrame(
        geometry=[box(min(P["lon_min"], P["lon_max"]),
                      min(P["lat_min"], P["lat_max"]),
                      max(P["lon_min"], P["lon_max"]),
                      max(P["lat_min"], P["lat_max"]))],
        crs=4326
    )
    xmin, ymin, xmax, ymax = aoi_ll_zoom.to_crs(utm_plot).total_bounds
else:
    bounds = []
    if not b_utm.empty:       bounds.append(b_utm.total_bounds)
    if not buffers_utm.empty: bounds.append(buffers_utm.total_bounds)
    if aoi_utm is not None and not aoi_utm.empty: bounds.append(aoi_utm.total_bounds)
    xmin = min(b[0] for b in bounds); ymin = min(b[1] for b in bounds)
    xmax = max(b[2] for b in bounds); ymax = max(b[3] for b in bounds)

# Plot
fig, ax = plt.subplots(figsize=P["FIGSIZE"])
legend_elems = []

# Buffers
buffers_utm.plot(ax=ax, facecolor=P["BUF_FACE"], edgecolor=P["BUF_EDGE"],
                 alpha=P["BUF_ALPHA"], zorder=1)
legend_elems.append(Patch(facecolor=P["BUF_FACE"], edgecolor=P["BUF_EDGE"],
                          alpha=P["BUF_ALPHA"], label=P["LBL_BUFFERS"]))

# All buildings
b_utm.plot(ax=ax, facecolor=P["BLDG_FACE"], edgecolor=P["BLDG_EDGE"],
           linewidth=P["BLDG_LW"], alpha=P["BLDG_ALPHA"], zorder=2)
legend_elems.append(Patch(facecolor=P["BLDG_FACE"], edgecolor=P["BLDG_EDGE"],
                          label=P["LBL_BUILDINGS"]))

# Damaged buildings
dam_utm.plot(ax=ax, facecolor=P["DAM_FACE"], edgecolor=P["DAM_EDGE"],
             linewidth=P["DAM_LW"], alpha=P["DAM_ALPHA"], zorder=3)
legend_elems.append(Patch(facecolor=P["DAM_FACE"], edgecolor=P["DAM_EDGE"],
                          label=P["LBL_DAMAGED"]))

# AOI boundary
if P["DRAW_AOI"] and aoi_utm is not None and not aoi_utm.empty:
    aoi_utm.boundary.plot(ax=ax, color=P["AOI_COLOR"], linewidth=P["AOI_LW"], zorder=4)
    legend_elems.append(Line2D([0],[0], color=P["AOI_COLOR"], lw=P["AOI_LW"], label=P["LBL_AOI"]))

# Frame + decor
ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

if P["ADD_NORTH_ARROW"]:
    ax.annotate(P["NA_LABEL"], xy=(P["NA_X"], P["NA_Y"]), xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=P["NA_FONTSIZE"], fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                clip_on=False, zorder=20)

if P["ADD_SCALEBAR"]:
    ax.add_artist(ScaleBar(1, P["SB_UNITS"], location=P["SB_LOC"]))

ax.set_aspect("equal"); ax.ticklabel_format(style="plain")
ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
ax.set_title(P["TITLE"])

ax.legend(handles=legend_elems, loc=P["LEGEND_LOC"], frameon=True)

# Save + show
out_png = Path(IMAGES_DIR) / P["OUT_NAME"]
fig.savefig(out_png, dpi=P["DPI"], bbox_inches="tight")
plt.close(fig)
print(f" Saved: {out_png}")
display(Image(filename=str(out_png)))


In [ ]:
# =============================== E9) Summary ===============================
n_eval   = len(edges_eval) if 'edges_eval' in globals() else 0
n_dmg    = len(edges_dmg)  if 'edges_dmg'  in globals() else 0
n_ok     = len(edges_ok)   if 'edges_ok'   in globals() else 0
n_island = len(buffer_islands) if 'buffer_islands' in globals() else 0
avg_buf  = float(per_buffer_gdf["buf_m"].mean()) if ('per_buffer_gdf' in globals() and "buf_m" in per_buffer_gdf.columns) else float("nan")

print("\n===== Section E Summary =====")
print(f"Total edges evaluated:        {n_eval:,}")
print(f"  Damaged edges (touching):   {n_dmg:,}")
print(f"  Undamaged edges:            {n_ok:,}")
print(f"Buffer islands (dissolved):   {n_island:,}")
print(f"Per-building buffer (m):      mean ≈ {avg_buf:.2f}")
print("\nOutputs:")
print(f"  Evaluated_Edges:            {EVAL_EDGES_SHP}")
print(f"  Damaged_Edges:              {DAMAGED_EDGES_SHP}")
print(f"  Undamaged_Edges:            {UNDAMAGED_EDGES_SHP}")
print(f"  Buffer islands (QA, gpkg):  {BUFFER_ISLANDS_GPKG}")
print(f"  Per-building buffers (gpkg):{PER_BUILDING_BUFF_GPKG}")
print(f"  Figure:                     {Path(IMAGES_DIR) / 'Buffer_Zones.png'}")
print("========================================")


## F) Tunnels and Bridges Damage Evaluation

In [ ]:
# =============================== F0) Imports ===============================
import os
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio.features import shapes
from shapely.geometry import shape, box
from shapely.prepared import prep
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from IPython.display import Image, display
import yaml

In [ ]:
# ====================== F1) User Options & Paths ======================
OUT_BASE   = r"./Outputs"
A_OUT_DIR  = os.path.join(OUT_BASE, "A")
B_OUT_DIR  = os.path.join(OUT_BASE, "B")
C_OUT_DIR  = os.path.join(OUT_BASE, "C")
E_OUT_DIR  = os.path.join(OUT_BASE, "E")
F_OUT_DIR  = os.path.join(OUT_BASE, "F")
IMAGES_DIR = os.path.join(OUT_BASE, "Images")
os.makedirs(F_OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

# --- Data ---
# Evaluated edges (with Damaged yes/no) from Section E
if 'edges_eval' in globals() and isinstance(edges_eval, gpd.GeoDataFrame) and not edges_eval.empty:
    edges_eval_src = edges_eval.copy()
elif 'EVAL_EDGES_SHP' in globals() and Path(EVAL_EDGES_SHP).exists():
    edges_eval_src = gpd.read_file(EVAL_EDGES_SHP)
else:
    _eval_e = Path(E_OUT_DIR) / "Evaluated_Edges.shp"
    if _eval_e.exists():
        edges_eval_src = gpd.read_file(_eval_e)
    else:
        raise RuntimeError("Evaluated edges not found. Run Section E first.")

# Damaged pixels vector
if 'damaged_gdf' in globals() and isinstance(damaged_gdf, gpd.GeoDataFrame) and not damaged_gdf.empty:
    dmg_vec_pref = damaged_gdf.copy()
else:
    dmg_vec_pref = None

# Binary damaged mask GeoTIFF from Section B
DAMAGED_MASK_TIF = globals().get("DAMAGED_MASK_TIF", Path(C_OUT_DIR) / "AOI_DPM_DamagedMask_ge84.tif")

# --- Output file paths ---
EVAL_EDGES_TB_SHP       = Path(F_OUT_DIR) / "Evaluated_Edges_withTunnels_Bridges.shp"
DAMAGED_EDGES_TB_SHP    = Path(F_OUT_DIR) / "Damaged_Edges_withTunnels_Bridges.shp"
UNDAMAGED_EDGES_TB_SHP  = Path(F_OUT_DIR) / "Undamaged_Edges_withTunnels_Bridges.shp"


In [ ]:
# =========== F2) Damaged pixels vector memory ===========
if dmg_vec_pref is not None and not dmg_vec_pref.empty:
    dmg_vec = dmg_vec_pref.to_crs(edges_eval_src.crs) if dmg_vec_pref.crs != edges_eval_src.crs else dmg_vec_pref.copy()
else:
    if not (isinstance(DAMAGED_MASK_TIF, (str, Path)) and Path(DAMAGED_MASK_TIF).exists()):
        print("No damaged pixels available; skipping TB override.")
        edges_eval_tb = edges_eval_src.copy()
        edges_dmg_tb  = edges_eval_tb.loc[edges_eval_tb["Damaged"].astype(str).str.lower().eq("yes")].copy()
        edges_ok_tb   = edges_eval_tb.loc[edges_eval_tb["Damaged"].astype(str).str.lower().eq("no")].copy()

        edges_eval_tb.to_file(EVAL_EDGES_TB_SHP)
        gpd.GeoDataFrame(geometry=edges_dmg_tb.geometry, crs=edges_dmg_tb.crs).to_file(DAMAGED_EDGES_TB_SHP)
        gpd.GeoDataFrame(geometry=edges_ok_tb.geometry,  crs=edges_ok_tb.crs).to_file(UNDAMAGED_EDGES_TB_SHP)

        print(f" Evaluated edges: {len(edges_eval_tb)}")
        print(f"  Damaged   (buffer OR TB override): {len(edges_dmg_tb)}")
        print(f"  Undamaged:                         {len(edges_ok_tb)}")
        print(" Saved:\n   ", EVAL_EDGES_TB_SHP, "\n   ", DAMAGED_EDGES_TB_SHP, "\n   ", UNDAMAGED_EDGES_TB_SHP)
    else:
        with rio.open(DAMAGED_MASK_TIF) as ds:
            arr = ds.read(1)
            t   = ds.transform
            mask = (arr == 1)
            if not mask.any():
                dmg_vec = gpd.GeoDataFrame(geometry=[], crs=ds.crs)
            else:
                dam_polys = [shape(geom) for geom, val in shapes(mask.astype(np.uint8), mask=mask, transform=t) if val == 1]
                dmg_vec = gpd.GeoDataFrame(geometry=dam_polys, crs=ds.crs)

        if dmg_vec.crs is not None and dmg_vec.crs != edges_eval_src.crs:
            dmg_vec = dmg_vec.to_crs(edges_eval_src.crs)


In [ ]:
# =========== F3) Identify tunnel/bridge edges (robust yes/no parsing) ===========
def _yes_like(series):
    s = series.astype(str).str.lower().str.strip()
    return s.isin(["yes", "true", "1", "y", "t"])

has_bridge = _yes_like(edges_eval_src["bridge"]) if "bridge" in edges_eval_src.columns else gpd.pd.Series(False, index=edges_eval_src.index)
has_tunnel = _yes_like(edges_eval_src["tunnel"]) if "tunnel" in edges_eval_src.columns else gpd.pd.Series(False, index=edges_eval_src.index)
tb_mask = has_bridge | has_tunnel
tb_edges = edges_eval_src.loc[tb_mask].copy()


In [ ]:
# ======= F4) TB ∩ damage intersects → indices to force as Damaged =======
if dmg_vec is None or dmg_vec.empty or tb_edges.empty:
    print("No TB edges and/or no damaged pixels; using original edge classifications.")
    tb_hit_idx = gpd.pd.Index([])
else:
    try:
        inv_tb = ~tb_edges.is_valid
        if inv_tb.any():
            tb_edges.loc[inv_tb, "geometry"] = tb_edges.loc[inv_tb, "geometry"].buffer(0)
        inv_d = ~dmg_vec.is_valid
        if inv_d.any():
            dmg_vec.loc[inv_d, "geometry"] = dmg_vec.loc[inv_d, "geometry"].buffer(0)
    except Exception:
        pass

    _ = dmg_vec.sindex
    hits = gpd.sjoin(tb_edges[["geometry"]], dmg_vec[["geometry"]], how="inner", predicate="intersects")
    tb_hit_idx = hits.index.unique()

print(f"Tunnel/Bridge edges total: {len(tb_edges)} | intersecting damaged pixels: {len(tb_hit_idx)}")


In [ ]:
# ================== F5) Apply overrides → split → save ==================
edges_eval_tb = edges_eval_src.copy()
if len(tb_hit_idx) > 0:
    edges_eval_tb.loc[tb_hit_idx, "Damaged"] = "yes"

edges_dmg_tb = edges_eval_tb.loc[edges_eval_tb["Damaged"].astype(str).str.lower().eq("yes")].copy()
edges_ok_tb  = edges_eval_tb.loc[edges_eval_tb["Damaged"].astype(str).str.lower().eq("no")].copy()

# Save
edges_eval_tb.to_file(EVAL_EDGES_TB_SHP)
gpd.GeoDataFrame(geometry=edges_dmg_tb.geometry, crs=edges_dmg_tb.crs).to_file(DAMAGED_EDGES_TB_SHP)
gpd.GeoDataFrame(geometry=edges_ok_tb.geometry,  crs=edges_ok_tb.crs).to_file(UNDAMAGED_EDGES_TB_SHP)

print(f" Evaluated edges: {len(edges_eval_tb)}")
print(f"  Damaged   (buffer OR TB override): {len(edges_dmg_tb)}")
print(f"  Undamaged:                         {len(edges_ok_tb)}")
print(" Saved:")
print("   ", EVAL_EDGES_TB_SHP)
print("   ", DAMAGED_EDGES_TB_SHP)
print("   ", UNDAMAGED_EDGES_TB_SHP)


In [ ]:
# ================== F6) Plot: Damaged Edges, TB, Buildings ==================
P = {
    # ---- Output ----
    "OUT_NAME": "Damaged_Edges_Tunnels_Bridges.png",  # output PNG name (saved to IMAGES_DIR)

    # ---- Manual zoom in EPSG:4326 (set all to None for auto extent) ----
    "lon_min": 36.900,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.930,             # max longitude (None = auto)
    "lat_min": 37.570,             # min latitude  (None = auto)
    "lat_max": 37.600,             # max latitude  (None = auto)

    # ---- Figure & labels ----
    "FIGSIZE": (11, 10),    # figure size (w, h) inches
    "DPI": 300,             # export resolution
    "TITLE": "Damaged Edges, Tunnels/Bridges, and Buildings",
    "XLABEL": "Easting (m)",   # projected meters
    "YLABEL": "Northing (m)",  # projected meters

    # ---- Base edges (all) ----
    "EDGE_ALL_COLOR": "#9e9e9e",  # color for all edges
    "EDGE_ALL_LW": 0.6,           # linewidth
    "EDGE_ALL_ALPHA": 1.0,        # transparency
    "LBL_EDGE_ALL": "All edges",  # legend label

    # ---- Damaged edges ----
    "EDGE_DMG_COLOR": "black",
    "EDGE_DMG_LW": 1.0,
    "EDGE_DMG_ALPHA": 1.0,
    "LBL_EDGE_DMG": "Damaged edges",

    # ---- Damaged tunnels/bridges subset ----
    "TB_COLOR": "#1f78b4",
    "TB_LW": 1.6,
    "TB_ALPHA": 1.0,
    "LBL_TB": "Damaged tunnels/bridges",

    # ---- Buildings (all) ----
    "BLDG_FACE": "white",
    "BLDG_EDGE": "black",
    "BLDG_LW": 0.3,
    "BLDG_ALPHA": 1.0,
    "LBL_BLDG": "Buildings",

    # ---- Damaged buildings ----
    "BLDG_DAM_FACE": "#d73027",
    "BLDG_DAM_EDGE": "black",
    "BLDG_DAM_LW": 0.2,
    "BLDG_DAM_ALPHA": 1.0,
    "LBL_BLDG_DAM": "Damaged buildings",

    # ---- AOI boundary (optional if provided upstream) ----
    "DRAW_AOI": True,
    "AOI_COLOR": "black",
    "AOI_LW": 1.0,
    "LBL_AOI": "AOI boundary",

    # ---- North arrow & scalebar ----
    "ADD_NORTH_ARROW": True,
    "NA_X": 0.05, "NA_Y": 0.15,   # arrow position (axes fraction)
    "NA_LEN": 0.08,               # arrow length (axes fraction)
    "NA_LABEL": "N",
    "NA_COLOR": "black", "NA_LW": 2, "NA_FONTSIZE": 14,
    "ADD_SCALEBAR": True,         # scalebar
    "SB_UNITS": "m",
    "SB_LOC": "lower right",      # scalebar location

    # ---- Legend ----
    "LEGEND_LOC": "upper right"   # legend placement
}
# ------------------------------------------------------------------------------

# Edges (prefer TB-evaluated, then evaluated, then basic evaluated set)
if 'edges_eval_tb' in globals() and isinstance(edges_eval_tb, gpd.GeoDataFrame) and not edges_eval_tb.empty:
    e_src = edges_eval_tb.copy()
elif 'edges_eval_src' in globals() and isinstance(edges_eval_src, gpd.GeoDataFrame) and not edges_eval_src.empty:
    e_src = edges_eval_src.copy()
elif EVAL_EDGES_TB_SHP.exists():
    e_src = gpd.read_file(EVAL_EDGES_TB_SHP)
else:
    e_src = gpd.read_file(EVAL_EDGES_SHP) if 'EVAL_EDGES_SHP' in globals() and Path(EVAL_EDGES_SHP).exists() else None
    if e_src is None:
        raise RuntimeError("No evaluated edges available for plotting.")
if "Damaged" not in e_src.columns:
    raise ValueError("Edges layer must include 'Damaged'.")

# Buildings (must have Damaged)
if 'b' in globals() and isinstance(b, gpd.GeoDataFrame) and not b.empty and ("Damaged" in b.columns):
    b_src = b.copy()
elif 'buildings_r' in globals() and isinstance(buildings_r, gpd.GeoDataFrame) and not buildings_r.empty and ("Damaged" in buildings_r.columns):
    b_src = buildings_r.copy()
elif 'EVAL_OUT_SHP' in globals() and Path(EVAL_OUT_SHP).exists():
    b_src = gpd.read_file(EVAL_OUT_SHP)
    if "Damaged" not in b_src.columns:
        raise ValueError("Loaded buildings file has no 'Damaged' column.")
else:
    _b_eval = Path(B_OUT_DIR) / "Evaluated_Buildings.shp"
    if not _b_eval.exists():
        raise RuntimeError("No evaluated buildings found for plotting.")
    b_src = gpd.read_file(_b_eval)

aoi_src = aoi_gdf_ll if 'aoi_gdf_ll' in globals() else None

# Common local UTM
seed   = b_src if not b_src.empty else e_src
utm_crs = seed.to_crs(4326).estimate_utm_crs()
e_utm   = e_src.to_crs(utm_crs)
b_utm   = b_src.to_crs(utm_crs)
aoi_utm = aoi_src.to_crs(utm_crs) if isinstance(aoi_src, gpd.GeoDataFrame) else None

# Subsets
edge_is_dmg   = e_utm["Damaged"].astype(str).str.lower().eq("yes")
edges_all_utm = e_utm
edges_dmg_utm = e_utm[edge_is_dmg].copy()

def _yes_like(series):
    s = series.astype(str).str.lower().str.strip()
    return s.isin(["yes", "true", "1", "y", "t"])

has_bridge = _yes_like(e_utm["bridge"]) if "bridge" in e_utm.columns else gpd.pd.Series(False, index=e_utm.index)
has_tunnel = _yes_like(e_utm["tunnel"]) if "tunnel" in e_utm.columns else gpd.pd.Series(False, index=e_utm.index)
tb_dmg_utm = e_utm[(edge_is_dmg) & (has_bridge | has_tunnel)].copy()

b_dmg = b_utm[b_utm["Damaged"].astype(str).str.lower().eq("yes")].copy()

# Optional lon/lat zoom → UTM frame
use_bbox = all(P[k] is not None for k in ("lon_min","lon_max","lat_min","lat_max"))
if use_bbox:
    aoi_ll_zoom = gpd.GeoDataFrame(
        geometry=[box(min(P["lon_min"], P["lon_max"]),
                      min(P["lat_min"], P["lat_max"]),
                      max(P["lon_min"], P["lon_max"]),
                      max(P["lat_min"], P["lat_max"]))],
        crs=4326
    )
    xmin, ymin, xmax, ymax = aoi_ll_zoom.to_crs(utm_crs).total_bounds
else:
    bounds = []
    if not edges_all_utm.empty: bounds.append(edges_all_utm.total_bounds)
    if not b_utm.empty:         bounds.append(b_utm.total_bounds)
    if aoi_utm is not None and not aoi_utm.empty: bounds.append(aoi_utm.total_bounds)
    xmin = min(b[0] for b in bounds); ymin = min(b[1] for b in bounds)
    xmax = max(b[2] for b in bounds); ymax = max(b[3] for b in bounds)

# Plot
fig, ax = plt.subplots(figsize=P["FIGSIZE"])
legend_items = []

# All edges
edges_all_utm.plot(ax=ax, color=P["EDGE_ALL_COLOR"], linewidth=P["EDGE_ALL_LW"],
                   alpha=P["EDGE_ALL_ALPHA"], zorder=1)
legend_items.append(Line2D([0],[0], color=P["EDGE_ALL_COLOR"], lw=2.5, label=P["LBL_EDGE_ALL"]))

# Buildings (all)
b_utm.plot(ax=ax, facecolor=P["BLDG_FACE"], edgecolor=P["BLDG_EDGE"],
           linewidth=P["BLDG_LW"], alpha=P["BLDG_ALPHA"], zorder=2)
legend_items.append(Patch(facecolor=P["BLDG_FACE"], edgecolor=P["BLDG_EDGE"], label=P["LBL_BLDG"]))

# Damaged buildings
b_dmg.plot(ax=ax, facecolor=P["BLDG_DAM_FACE"], edgecolor=P["BLDG_DAM_EDGE"],
           linewidth=P["BLDG_DAM_LW"], alpha=P["BLDG_DAM_ALPHA"], zorder=3)
legend_items.append(Patch(facecolor=P["BLDG_DAM_FACE"], edgecolor=P["BLDG_DAM_EDGE"], label=P["LBL_BLDG_DAM"]))

# Damaged edges
edges_dmg_utm.plot(ax=ax, color=P["EDGE_DMG_COLOR"], linewidth=P["EDGE_DMG_LW"],
                   alpha=P["EDGE_DMG_ALPHA"], zorder=4)
legend_items.append(Line2D([0],[0], color=P["EDGE_DMG_COLOR"], lw=3, label=P["LBL_EDGE_DMG"]))

# Damaged tunnels/bridges
tb_dmg_utm.plot(ax=ax, color=P["TB_COLOR"], linewidth=P["TB_LW"],
                alpha=P["TB_ALPHA"], zorder=5)
legend_items.append(Line2D([0],[0], color=P["TB_COLOR"], lw=3, label=P["LBL_TB"]))

# AOI boundary (optional)
if P["DRAW_AOI"] and aoi_utm is not None and not aoi_utm.empty:
    aoi_utm.boundary.plot(ax=ax, color=P["AOI_COLOR"], linewidth=P["AOI_LW"], zorder=6)
    legend_items.append(Line2D([0],[0], color=P["AOI_COLOR"], lw=P["AOI_LW"], label=P["LBL_AOI"]))

# Frame + decor
ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

# North arrow
if P["ADD_NORTH_ARROW"]:
    ax.annotate(P["NA_LABEL"], xy=(P["NA_X"], P["NA_Y"]), xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=P["NA_FONTSIZE"], fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                clip_on=False, zorder=20)

# Scalebar
if P["ADD_SCALEBAR"]:
    ax.add_artist(ScaleBar(1, P["SB_UNITS"], location=P["SB_LOC"]))

ax.set_aspect("equal"); ax.ticklabel_format(style="plain")
ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
ax.set_title(P["TITLE"])

# Legend
ax.legend(handles=legend_items, loc=P["LEGEND_LOC"], frameon=True)

# Save + show
out_png = Path(IMAGES_DIR) / P["OUT_NAME"]
fig.savefig(out_png, dpi=P["DPI"], bbox_inches="tight")
plt.close(fig)
print(f" Saved: {out_png}")
display(Image(filename=str(out_png)))


In [ ]:
# =============================== F7) Summary ===============================
n_eval = len(edges_eval_tb) if 'edges_eval_tb' in globals() else len(edges_eval_src)
n_dmg  = len(edges_dmg_tb)  if 'edges_dmg_tb'  in globals() else int((edges_eval_src["Damaged"].astype(str).str.lower()=="yes").sum())
n_ok   = len(edges_ok_tb)   if 'edges_ok_tb'   in globals() else int((edges_eval_src["Damaged"].astype(str).str.lower()=="no").sum())
n_tb   = int(tb_edges.shape[0]) if 'tb_edges' in globals() else 0
n_tb_hit = int(len(tb_hit_idx)) if 'tb_hit_idx' in globals() else 0

print("\n===== Section F Summary =====")
print(f"Evaluated edges (total):                 {n_eval:,}")
print(f"  Damaged edges (after TB override):     {n_dmg:,}")
print(f"  Undamaged edges:                       {n_ok:,}")
print(f"Tunnel/Bridge edges (OSM):               {n_tb:,}")
print(f"  TB edges intersecting damaged pixels:  {n_tb_hit:,}")
print("Outputs:")
print(f"  Evaluated_Edges_withTunnels_Bridges:   {EVAL_EDGES_TB_SHP}")
print(f"  Damaged_Edges_withTunnels_Bridges:     {DAMAGED_EDGES_TB_SHP}")
print(f"  Undamaged_Edges_withTunnels_Bridges:   {UNDAMAGED_EDGES_TB_SHP}")
print(f"  Figure:                                 {Path(IMAGES_DIR) / 'Damaged_Edges_Tunnels_Bridges.png'}")
print("=========================================================")


## G) Damaged Node Calculation

In [ ]:
# =============================== G0) Imports ===============================
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from shapely.geometry import box
from IPython.display import Image, display
import yaml


In [ ]:
# ====================== G1) User Options & Paths ======================
OUT_BASE   = r"./Outputs"
A_OUT_DIR  = os.path.join(OUT_BASE, "A")
E_OUT_DIR  = os.path.join(OUT_BASE, "E")
F_OUT_DIR  = os.path.join(OUT_BASE, "F")
G_OUT_DIR  = os.path.join(OUT_BASE, "G")
IMAGES_DIR = os.path.join(OUT_BASE, "Images")
os.makedirs(G_OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

# Preferred evaluated-edges data
EDGE_VARS_MEM = ["edges_eval_tb", "edges_eval_src", "edges_eval"]
EDGE_PATHS_FS = [os.path.join(F_OUT_DIR, "Evaluated_Edges_withTunnels_Bridges.shp"),
                 os.path.join(E_OUT_DIR, "Evaluated_Edges.shp")]

# Preferred nodes data
NODES_VAR_MEM = "nodes"
NODES_PATH_FS = os.path.join(A_OUT_DIR, "osm_nodes.shp")

# Output files
EVAL_NODES_TB_SHP   = Path(G_OUT_DIR) / "Evaluated_Nodes_withTunnels_Bridges.shp"
DAMAGED_NODES_TB_SHP= Path(G_OUT_DIR) / "Damaged_Nodes_withTunnels_Bridges.shp"
UNDAM_NODES_TB_SHP  = Path(G_OUT_DIR) / "Undamaged_Nodes_withTunnels_Bridges.shp"


In [ ]:
# ====================== G2) Load Data (edges & nodes) ======================

# 1) Edges with 'Damaged'
edges_eval_candidates = []

# --- From in-memory variables ---
for var in EDGE_VARS_MEM:
    if var in globals():
        g = globals()[var]
        if isinstance(g, gpd.GeoDataFrame) and not g.empty and ("Damaged" in g.columns):
            edges_eval_candidates.append(g.copy())

# --- From filesystem paths ---
for p in EDGE_PATHS_FS:
    if Path(p).exists():
        g = gpd.read_file(p)
        if not g.empty and ("Damaged" in g.columns):
            edges_eval_candidates.append(g)

if not edges_eval_candidates:
    raise RuntimeError("No evaluated edges with a 'Damaged' column found.")

edges_eval_in = edges_eval_candidates[0]

def ensure_uv_columns(edge_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Ensure columns 'u' and 'v' exist. If MultiIndex (u,v,key), reset index and map to 'u'/'v'.
    """
    if edge_gdf.empty:
        return edge_gdf
    # If 'u' and 'v' already exist, just return
    if {'u', 'v'}.issubset(edge_gdf.columns):
        return edge_gdf
    # If MultiIndex, try to interpret first two levels as u, v
    if isinstance(edge_gdf.index, pd.MultiIndex):
        df = edge_gdf.reset_index()
        idx_names = list(edge_gdf.index.names)
        rename_map = {}
        if len(idx_names) >= 1:
            rename_map[idx_names[0] if idx_names[0] else 'level_0'] = 'u'
        if len(idx_names) >= 2:
            rename_map[idx_names[1] if idx_names[1] else 'level_1'] = 'v'
        df = df.rename(columns=rename_map)
        if 'u' not in df.columns or 'v' not in df.columns:
            raise KeyError("Could not map MultiIndex index levels to 'u'/'v'. Inspect edges.")
        return df
    raise KeyError("Edges lack 'u'/'v' columns and are not a MultiIndex—inspect edges schema.")

edges_u = ensure_uv_columns(edges_eval_in)

# 2) Nodes (OSM)
if NODES_VAR_MEM in globals() and isinstance(globals()[NODES_VAR_MEM], gpd.GeoDataFrame) and not globals()[NODES_VAR_MEM].empty:
    nodes_src = globals()[NODES_VAR_MEM].copy()
elif Path(NODES_PATH_FS).exists():
    nodes_src = gpd.read_file(NODES_PATH_FS)
else:
    raise RuntimeError("No OSM nodes found (memory or Outputs_Final\\A).")

# Ensure node index matches edge u/v IDs (use 'osmid' as index if present)
if "osmid" in nodes_src.columns:
    nodes_src = nodes_src.set_index("osmid", drop=False)
else:
    raise RuntimeError("nodes_src has no 'osmid' column; cannot align with edge u/v node IDs.")


In [ ]:
# ============== G3) Classify nodes: all-incident-edges-damaged rule ==============
# Normalize 'Damaged' yes/no on edges
edge_dam = edges_u["Damaged"].astype(str).str.lower().str.strip().eq("yes")
edges_dmg = edges_u[edge_dam].copy()

print(f" Evaluated edges (data): {len(edges_u):,}")
print(f"   Damaged edges:           {len(edges_dmg):,}")
print(f"   Undamaged edges:         {len(edges_u) - len(edges_dmg):,}")

def node_degree_counts(edge_gdf: gpd.GeoDataFrame) -> pd.Series:
    if edge_gdf.empty:
        return pd.Series(dtype=int)
    # degree = number of incident edges (counts of u and v)
    return pd.concat([edge_gdf['u'], edge_gdf['v']]).value_counts()

deg_total   = node_degree_counts(edges_u)
deg_damaged = node_degree_counts(edges_dmg)

deg_df = (
    pd.DataFrame({'deg_total': deg_total})
      .join(deg_damaged.rename('deg_damaged'), how='left')
      .fillna({'deg_damaged': 0})
      .astype(int)
)

# Rule: node is "yes" only if ALL incident edges are damaged (and degree ≥ 1)
node_is_damaged = (deg_df['deg_damaged'] == deg_df['deg_total']) & (deg_df['deg_total'] >= 1)
deg_df['Damaged'] = node_is_damaged.map({True: "yes", False: "no"})

# Join onto nodes (index should be node IDs)
nodes_eval = nodes_src.join(deg_df, how='left')

# Isolated nodes (no incident edges) → undamaged by definition
nodes_eval[['deg_total','deg_damaged']] = nodes_eval[['deg_total','deg_damaged']].fillna(0).astype(int)
nodes_eval['Damaged'] = nodes_eval['Damaged'].fillna("no").astype(str)

nodes_dmg = nodes_eval[nodes_eval['Damaged'].str.lower().eq("yes")].copy()
nodes_ok  = nodes_eval[nodes_eval['Damaged'].str.lower().ne("yes")].copy()

print(f" Evaluated nodes:                        {len(nodes_eval):,}")
print(f"   Damaged nodes (all incident damaged):  {len(nodes_dmg):,}")
print(f"   Undamaged nodes:                       {len(nodes_ok):,}")


In [ ]:
# =============================== G4) Save nodes ===============================
def _safe_save_nodes(gdf: gpd.GeoDataFrame, path: str):
    """Reset index to avoid duplicate 'osmid' when writing to file."""
    if gdf is None or gdf.empty:
        return
    gdf_out = gdf.reset_index(drop=True)
    gdf_out.to_file(path)

_safe_save_nodes(nodes_eval, EVAL_NODES_TB_SHP)
_safe_save_nodes(nodes_dmg, DAMAGED_NODES_TB_SHP)
_safe_save_nodes(nodes_ok,  UNDAM_NODES_TB_SHP)

print(" Saved:")
print("  ", EVAL_NODES_TB_SHP)
print("  ", DAMAGED_NODES_TB_SHP)
print("  ", UNDAM_NODES_TB_SHP)


In [ ]:
# ======= G5) Plot: Nodes  =======
P = {
    # ---- Output ----
    "OUT_NAME": "Damaged_Nodes.png",      # output PNG name (saved to IMAGES_DIR)

    # ---- Manual zoom in EPSG:4326 (set all to None for auto extent) ----
    "lon_min": 36.900,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.930,             # max longitude (None = auto)
    "lat_min": 37.570,             # min latitude  (None = auto)
    "lat_max": 37.600,             # max latitude  (None = auto)

    # ---- Figure & labels ----
    "FIGSIZE": (11, 10),                  # figure size (w, h) inches
    "DPI": 300,                           # export resolution
    "TITLE": "Damaged Nodes (black) & Edges (red) with Network Context",
    "XLABEL": "Easting (m)",              # projected meters
    "YLABEL": "Northing (m)",             # projected meters

    # ---- Edges style ----
    "EDGE_ALL_COLOR": "#b0b0b0",          # undamaged edges color (gray)
    "EDGE_ALL_LW": 0.6,                   # linewidth
    "EDGE_ALL_ALPHA": 1.0,                # transparency
    "EDGE_DMG_COLOR": "#d73027",          # damaged edges color (red)
    "EDGE_DMG_LW": 1.2,
    "EDGE_DMG_ALPHA": 1.0,
    "LBL_EDGE_OK": "Undamaged edges",     # legend label
    "LBL_EDGE_DMG": "Damaged edges",      # legend label

    # ---- Nodes style ----
    "NODE_OK_COLOR": "#1f78b4",           # undamaged nodes (blue)
    "NODE_OK_SIZE": 12,
    "NODE_OK_ALPHA": 1.0,
    "NODE_DMG_COLOR": "black",            # damaged nodes (black)
    "NODE_DMG_SIZE": 18,
    "NODE_DMG_ALPHA": 1.0,
    "LBL_NODE_OK": "Undamaged nodes",     # legend label
    "LBL_NODE_DMG": "Damaged nodes",      # legend label

    # ---- AOI boundary (optional if provided upstream) ----
    "DRAW_AOI": True,
    "AOI_COLOR": "black",
    "AOI_LW": 1.0,

    # ---- North arrow & scalebar ----
    "ADD_NORTH_ARROW": True,
    "NA_X": 0.05, "NA_Y": 0.15,           # position (axes fraction)
    "NA_LEN": 0.08,                       # arrow length (axes fraction)
    "NA_LABEL": "N",
    "NA_COLOR": "black", "NA_LW": 2, "NA_FONTSIZE": 14,
    "ADD_SCALEBAR": True,                 # Scalebar
    "SB_UNITS": "m",
    "SB_LOC": "lower right",              # scalebar location

    # ---- Legend ----
    "LEGEND_LOC": "upper right"           # legend placement
}
# ------------------------------------------------------------------------------

# Reuse evaluated edges/nodes from memory in this section (unchanged logic)
e_src = edges_u.copy()
n_src = nodes_eval.copy()
aoi_src = aoi_gdf_ll if 'aoi_gdf_ll' in globals() else None

# Common CRS (local UTM from edges)
utm_crs   = e_src.to_crs(4326).estimate_utm_crs()
edges_utm = e_src.to_crs(utm_crs)
nodes_utm = n_src.to_crs(utm_crs)
aoi_utm   = aoi_src.to_crs(utm_crs) if isinstance(aoi_src, gpd.GeoDataFrame) else None

# Splits
dam_col = edges_utm["Damaged"].astype(str).str.lower().str.strip()
e_all   = edges_utm
e_dmg   = edges_utm[dam_col.eq("yes")].copy()
n_dmg   = nodes_utm[nodes_utm["Damaged"].astype(str).str.lower().eq("yes")].copy()
n_ok    = nodes_utm[nodes_utm["Damaged"].astype(str).str.lower().ne("yes")].copy()

# Optional lon/lat zoom → UTM frame
use_bbox = all(P[k] is not None for k in ("lon_min","lon_max","lat_min","lat_max"))
if use_bbox:
    aoi_ll_zoom = gpd.GeoDataFrame(
        geometry=[box(min(P["lon_min"], P["lon_max"]),
                      min(P["lat_min"], P["lat_max"]),
                      max(P["lon_min"], P["lon_max"]),
                      max(P["lat_min"], P["lat_max"]))],
        crs=4326
    )
    xmin, ymin, xmax, ymax = aoi_ll_zoom.to_crs(utm_crs).total_bounds
else:
    bounds = []
    if not e_all.empty:     bounds.append(e_all.total_bounds)
    if not nodes_utm.empty: bounds.append(nodes_utm.total_bounds)
    if aoi_utm is not None and not aoi_utm.empty: bounds.append(aoi_utm.total_bounds)
    xmin = min(b[0] for b in bounds); ymin = min(b[1] for b in bounds)
    xmax = max(b[2] for b in bounds); ymax = max(b[3] for b in bounds)

# Plot
fig, ax = plt.subplots(figsize=P["FIGSIZE"])
legend_items = []

# Edges (ok + damaged)
e_all.plot(ax=ax, color=P["EDGE_ALL_COLOR"], linewidth=P["EDGE_ALL_LW"],
           alpha=P["EDGE_ALL_ALPHA"], zorder=1)
legend_items.append(Line2D([0],[0], color=P["EDGE_ALL_COLOR"], lw=2, label=P["LBL_EDGE_OK"]))

e_dmg.plot(ax=ax, color=P["EDGE_DMG_COLOR"], linewidth=P["EDGE_DMG_LW"],
           alpha=P["EDGE_DMG_ALPHA"], zorder=2)
legend_items.append(Line2D([0],[0], color=P["EDGE_DMG_COLOR"], lw=2.5, label=P["LBL_EDGE_DMG"]))

# Nodes (ok + damaged)
if not n_ok.empty:
    n_ok.plot(ax=ax, color=P["NODE_OK_COLOR"], markersize=P["NODE_OK_SIZE"],
              alpha=P["NODE_OK_ALPHA"], zorder=3)
    legend_items.append(Line2D([0],[0], marker="o", linestyle="None",
                               markerfacecolor=P["NODE_OK_COLOR"], markeredgecolor="none",
                               markersize=8, label=P["LBL_NODE_OK"]))
if not n_dmg.empty:
    n_dmg.plot(ax=ax, color=P["NODE_DMG_COLOR"], markersize=P["NODE_DMG_SIZE"],
               alpha=P["NODE_DMG_ALPHA"], zorder=4)
    legend_items.append(Line2D([0],[0], marker="o", linestyle="None",
                               markerfacecolor=P["NODE_DMG_COLOR"], markeredgecolor="none",
                               markersize=8, label=P["LBL_NODE_DMG"]))

# AOI boundary (optional)
if P["DRAW_AOI"] and aoi_utm is not None and not aoi_utm.empty:
    aoi_utm.boundary.plot(ax=ax, color=P["AOI_COLOR"], linewidth=P["AOI_LW"], zorder=5)

# Frame + decor
ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

# North arrow
if P["ADD_NORTH_ARROW"]:
    ax.annotate(P["NA_LABEL"], xy=(P["NA_X"], P["NA_Y"]), xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=P["NA_FONTSIZE"], fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                clip_on=False, zorder=20)

# Scalebar
if P["ADD_SCALEBAR"]:
    ax.add_artist(ScaleBar(1, P["SB_UNITS"], location=P["SB_LOC"]))

ax.set_aspect("equal"); ax.ticklabel_format(style="plain")
ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
ax.set_title(P["TITLE"])

ax.legend(handles=legend_items, loc=P["LEGEND_LOC"], frameon=True)

out_png = Path(IMAGES_DIR) / P["OUT_NAME"]
fig.savefig(out_png, dpi=P["DPI"], bbox_inches="tight")
plt.close(fig)
print(f" Saved: {out_png}")
display(Image(filename=str(out_png)))


In [ ]:
# =============================== G6) Summary ===============================
n_edges_total = len(edges_u)
n_edges_dmg   = int(edge_dam.sum())
n_nodes_total = len(nodes_eval)
n_nodes_dmg   = len(nodes_dmg)
n_nodes_ok    = len(nodes_ok)

print("\n===== Section G Summary =====")
print(f"Edges evaluated (total):                 {n_edges_total:,}")
print(f"  Damaged edges:                         {n_edges_dmg:,}")
print(f"Nodes evaluated (total):                 {n_nodes_total:,}")
print(f"  Damaged nodes (all incident damaged):  {n_nodes_dmg:,}")
print(f"  Undamaged nodes:                       {n_nodes_ok:,}")
print("Outputs:")
print(f"  Evaluated Nodes:                       {EVAL_NODES_TB_SHP}")
print(f"  Damaged Nodes:                         {DAMAGED_NODES_TB_SHP}")
print(f"  Undamaged Nodes:                       {UNDAM_NODES_TB_SHP}")
print(f"  Figure:                                {Path(IMAGES_DIR) / 'Damaged_Nodes.png'}")
print("=======================================================")


## H) Utility 1: Shortest Path

In [ ]:
# =============================== H0) Imports ===============================
import os, json, math
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox

from shapely.geometry import Point, box
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from IPython.display import Image, display
import yaml
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
# ====================== H1) User Options & Paths ======================
# --- Origin/Destination in configuration
orig_node = optional("orig_node")                 # e.g., 157370362
dest_node = optional("dest_node")                 # e.g., 13081265302
orig_lonlat = tuple(require("orig_lonlat"))       # (lon, lat) used if node IDs are None
dest_lonlat = tuple(require("dest_lonlat"))

# How many distinct shortest routes to collect in configuration
ROUTE_TARGET = require("ROUTE_TARGET", int)

# --- Output ---
OUT_BASE   = r"./Outputs"
A_OUT_DIR  = os.path.join(OUT_BASE, "A")
E_OUT_DIR  = os.path.join(OUT_BASE, "E")
F_OUT_DIR  = os.path.join(OUT_BASE, "F")
H_OUT_DIR  = os.path.join(OUT_BASE, "H")
IMAGES_DIR = os.path.join(OUT_BASE, "Images")

SP_OUT_DIR = os.path.join(H_OUT_DIR, "Shortest_Path")
os.makedirs(SP_OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

print("Shortest-path outputs →", SP_OUT_DIR)


In [ ]:
# ======================== H2) Helper Functions ========================
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371008.8
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat/2)**2
         + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2)
    return 2 * R * math.asin(math.sqrt(a))

def ensure_nodes_index_xy(nodes_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    For routing:
      - Ensure index is 'osmid' (matching edge u/v IDs),
      - Ensure CRS is EPSG:4326 (lon/lat),
      - Force columns 'x' and 'y' to be geometry.x / geometry.y in that CRS.
    """
    n = nodes_gdf.copy()

    # --- Index: use OSMID if present ---
    if "osmid" in n.columns:
        if n.index.name != "osmid":
            n = n.set_index("osmid", drop=False)
    else:
        # fallback: create osmid from existing index
        if n.index.name != "osmid":
            n = n.reset_index().rename(columns={"index": "osmid"})
            n = n.set_index("osmid", drop=False)

    # --- CRS: make sure geometries are in WGS84 lon/lat ---
    if n.crs is None:
        n = n.set_crs(4326, allow_override=True)
    if n.crs.to_epsg() != 4326:
        n = n.to_crs(4326)

    # --- x, y: ALWAYS recompute from geometry (ignore any old x/y) ---
    n["x"] = n.geometry.x  # longitude
    n["y"] = n.geometry.y  # latitude

    return n

def ensure_edges_index(edges_gdf):
    e = edges_gdf.copy()
    for must in ("u","v"):
        if must not in e.columns:
            raise ValueError(f"Edges missing required column '{must}'.")
    if "key" not in e.columns:
        e["key"] = e.groupby(["u","v"]).cumcount()
    if (not isinstance(e.index, pd.MultiIndex)) or (list(e.index.names) != ["u","v","key"]):
        e = e.set_index(["u","v","key"], drop=False)
    drop_cols = [c for c in ("u","v","key") if c in e.columns]
    if drop_cols:
        e = e.drop(columns=drop_cols)
    return e

def ensure_lengths(edges_gdf):
    e = edges_gdf.copy()
    if "length" in e.columns and e["length"].notna().all():
        return e
    try:
        utm = e.estimate_utm_crs()
        e["length"] = e.to_crs(utm).geometry.length
    except Exception:
        def approx_len_m(geom):
            try:
                coords = list(geom.coords)
            except Exception:
                return 0.0
            s = 0.0
            for (x1,y1),(x2,y2) in zip(coords[:-1], coords[1:]):
                s += haversine_m(y1,x1,y2,x2)
            return s
        e["length"] = e.geometry.apply(approx_len_m)
    return e

def build_graph(nodes_gdf, edges_gdf):
    if nodes_gdf.crs is None:
        nodes_gdf = nodes_gdf.set_crs(4326, allow_override=True)
    if edges_gdf.crs is None:
        edges_gdf = edges_gdf.set_crs(nodes_gdf.crs, allow_override=True)
    return ox.convert.graph_from_gdfs(nodes_gdf, edges_gdf)

def nearest_node_from_lonlat(G, lon, lat):
    """
    Find the closest graph node to (lon, lat) using explicit haversine
    distance between your (lon, lat) and each node's (x, y) = (lon, lat).
    """
    best_n, best_d = None, float("inf")
    for n, d in G.nodes(data=True):
        x = d.get("x", None)  # longitude
        y = d.get("y", None)  # latitude
        if x is None or y is None:
            continue
        dist = haversine_m(lat, lon, y, x)  # (lat1, lon1, lat2, lon2)
        if dist < best_d:
            best_d, best_n = dist, n
    return best_n

def get_route_length(G, route):
    if not route or len(route) < 2:
        return 0.0
    total = 0.0
    for u, v in zip(route[:-1], route[1:]):
        data = G.get_edge_data(u, v)
        if not data:
            continue
        total += min(d.get("length", 0.0) for d in data.values())
    return total

def export_nodes_csv_rows(G, route):
    rows = []
    if not route:
        return rows
    try:
        step_lengths = ox.utils_graph.get_route_edge_attributes(G, route, "length")
    except Exception:
        step_lengths = []
        for u, v in zip(route[:-1], route[1:]):
            data = G.get_edge_data(u, v) or {}
            step_lengths.append(min(d.get("length", 0.0) for d in data.values()) if data else 0.0)
    cum = 0.0
    for seq, n in enumerate(route):
        if seq > 0 and seq - 1 < len(step_lengths):
            cum += step_lengths[seq - 1]
        lon = G.nodes[n].get("x"); lat = G.nodes[n].get("y")
        rows.append({"seq": seq, "node": n, "lon": lon, "lat": lat, "cum_length_m": cum})
    return rows

def _route_signature(route):
    return tuple(route) if route else tuple()

def add_north_arrow(ax, x=0.05, y=0.15, length=0.08):
    ax.annotate("N", xy=(x, y), xytext=(x, y - length),
                xycoords="axes fraction", textcoords="axes fraction",
                ha="center", va="center", fontsize=12, fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=2, color="black"),
                clip_on=False, zorder=20)


In [ ]:
# ========================= H3) Load Data =========================
if 'nodes' in globals() and isinstance(nodes, gpd.GeoDataFrame) and not nodes.empty:
    nodes_raw = nodes.copy()
else:
    nodes_path = Path(A_OUT_DIR) / "osm_nodes.shp"
    if nodes_path.exists():
        nodes_raw = gpd.read_file(nodes_path)

    else:
        raise RuntimeError("Nodes layer not found (memory or Outputs_Final\\A).")

# Evaluated edges
edges_eval_candidates = []
for _var in ("edges_eval_tb", "edges_eval_src", "edges_eval"):
    if _var in globals():
        _g = globals()[_var]
        if isinstance(_g, gpd.GeoDataFrame) and not _g.empty and ("Damaged" in _g.columns):
            edges_eval_candidates.append(_g.copy())

for _p in (Path(F_OUT_DIR) / "Evaluated_Edges_withTunnels_Bridges.shp",
           Path(E_OUT_DIR) / "Evaluated_Edges.shp"):
    if _p.exists():
        g = gpd.read_file(_p)
        if not g.empty and ("Damaged" in g.columns):
            edges_eval_candidates.append(g)

if not edges_eval_candidates:
    raise RuntimeError("Evaluated edges with 'Damaged' column not found (memory or disk).")

edges_eval_tb = edges_eval_candidates[0]

# Keep ONLY undamaged edges for routing
dam_col = edges_eval_tb["Damaged"].astype(str).str.lower().str.strip()
edges_ok = edges_eval_tb[~dam_col.eq("yes")].copy()
print(f"Routing over {len(edges_ok):,} undamaged edges (of {len(edges_eval_tb):,} total).")


In [ ]:
# ===================== H4) Normalize & Build Graph =====================
nodes_norm  = ensure_nodes_index_xy(nodes_raw)
edges_norm  = ensure_edges_index(edges_ok)
edges_norm  = ensure_lengths(edges_norm)

G = build_graph(nodes_norm, edges_norm)

# Resolve Origin /Destination nodes (ID first; else nearest by lon/lat)
if orig_node is None:
    orig_node = nearest_node_from_lonlat(G, *orig_lonlat)
if dest_node is None:
    dest_node = nearest_node_from_lonlat(G, *dest_lonlat)

orig_x = G.nodes[orig_node].get("x")
orig_y = G.nodes[orig_node].get("y")
dest_x = G.nodes[dest_node].get("x")
dest_y = G.nodes[dest_node].get("y")

print(f"Graph built → nodes: {G.number_of_nodes():,} | edges: {G.number_of_edges():,}")
print(f"Requested origin lon/lat:      {orig_lonlat}")
print(f"Requested destination lon/lat: {dest_lonlat}")
print(f"Snapped origin node:      {orig_node}  → ({orig_x:.6f}, {orig_y:.6f})")
print(f"Snapped destination node: {dest_node}  → ({dest_x:.6f}, {dest_y:.6f})")


In [ ]:
# =============== H5) Shortest Paths (expanding-radius variants) ===============
# First: direct shortest
try:
    route0 = ox.routing.shortest_path(G, orig_node, dest_node, weight="length")
except Exception:
    try:
        route0 = nx.shortest_path(G, orig_node, dest_node, weight="length", method="dijkstra")
    except Exception:
        route0 = None

routes_list, sigset = [], set()
if route0:
    routes_list.append(route0); sigset.add(_route_signature(route0))
    print(f"Direct path found: {len(route0)} nodes, {get_route_length(G, route0):.1f} m")
else:
    print("No direct path found yet—will try expanding-radius search.")

# Precompute single-source distances from origin
try:
    lengths, paths = nx.single_source_dijkstra(G, source=orig_node, weight="length")
except Exception as e:
    lengths, paths = {}, {}
    print(" single_source_dijkstra failed:", e)

# Node GeoDataFrame in lon/lat and meters for radius filtering
node_ids, node_pts_ll = [], []
for n, d in G.nodes(data=True):
    if "x" in d and "y" in d:
        node_ids.append(n); node_pts_ll.append(Point(d["x"], d["y"]))
nodes_ll = gpd.GeoDataFrame({"node": node_ids}, geometry=node_pts_ll, crs=4326)

try:
    utm = nodes_ll.estimate_utm_crs()
except Exception:
    utm = 3857
nodes_m = nodes_ll.to_crs(utm)

# Centers in meters
start_lon, start_lat = G.nodes[orig_node]["x"], G.nodes[orig_node]["y"]
dest_lon,  dest_lat  = G.nodes[dest_node]["x"],  G.nodes[dest_node]["y"]
start_m = gpd.GeoSeries([Point(start_lon, start_lat)], crs=4326).to_crs(utm).iloc[0]
dest_m  = gpd.GeoSeries([Point(dest_lon,  dest_lat)],  crs=4326).to_crs(utm).iloc[0]
d_m = start_m.distance(dest_m)
base_step = max(25.0, 0.01 * d_m)  # 1% of straight-line distance, min 25 m

has_sindex = getattr(nodes_m, "sindex", None) is not None
R = base_step
max_radius = max(d_m, 15000.0)
iters = 0

while len(routes_list) < ROUTE_TARGET and R <= max_radius:
    iters += 1
    circle = dest_m.buffer(R)

    if has_sindex:
        idx = list(nodes_m.sindex.query(circle.envelope))
        cand = nodes_m.iloc[idx]
        cand = cand[cand.geometry.within(circle)]
    else:
        cand = nodes_m[nodes_m.geometry.distance(dest_m) <= R]

    reachable = []
    if not cand.empty and lengths:
        reachable = [nid for nid in cand["node"].tolist() if nid in lengths]

    reachable_sorted = sorted(reachable, key=lambda nid: lengths[nid]) if reachable else []

    new_routes = 0
    for nid in reachable_sorted:
        rpath = paths.get(nid)
        if not rpath:
            continue
        sig = _route_signature(rpath)
        if sig in sigset:
            continue
        routes_list.append(rpath)
        sigset.add(sig)
        new_routes += 1
        if len(routes_list) >= ROUTE_TARGET:
            break

    pct = (R / d_m * 100.0) if d_m > 0 else float("inf")
    print(f"[Iter {iters}] R={R:.0f} m ({pct:.1f}%) | cand={0 if cand.empty else len(cand)} | "
          f"reach={len(reachable)} | new={new_routes} | total={len(routes_list)}")

    if len(routes_list) >= ROUTE_TARGET:
        break
    R += base_step

print(f"\n Total iterations: {iters}")
print(f" Routes collected: {len(routes_list)} (target {ROUTE_TARGET})")


In [ ]:
# =============== H6) Export CSV + PNGs ===============
P = {
    # ---- Output names ----
    "CSV_NAME": "All_Routes_Combined.csv",      # filename for the combined CSV of all routes
    "OUT_NAME_FMT": "Shortest_path_{i}.png",    # per-route PNG name; {i} will be replaced by route index

    # ---- Manual zoom in EPSG:4326 (set all to None for auto extent) ----
    "lon_min": 36.900,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.930,             # max longitude (None = auto)
    "lat_min": 37.570,             # min latitude  (None = auto)
    "lat_max": 37.600,             # max latitude  (None = auto)

    # ---- Figure & export ----
    "FIGSIZE": (7.5, 6.5),                      # figure size (width, height) in inches
    "DPI": 200,                                  # PNG resolution; lower = smaller files
    "XLABEL": "Easting (m)",                    # x-axis label
    "YLABEL": "Northing (m)",                   # y-axis label
    "LEGEND_LOC": "upper right",                # legend location (matplotlib position code)

    # ---- Base road network style ----
    "BASE_COLOR": "#cccccc",                    # base network line color
    "BASE_LW": 0.7,                             # base network line width
    "BASE_ALPHA": 1.0,                          # base network transparency (0..1)
    "LBL_BASE": "Road network",                 # legend label for base network

    # ---- AOI boundary (optional if provided upstream) ----
    "DRAW_AOI": True,                           # draw AOI boundary if available
    "AOI_COLOR": "black",                       # AOI boundary color
    "AOI_LW": 1.0,                              # AOI boundary line width
    "AOI_ALPHA": 1.0,                           # AOI boundary transparency
    "LBL_AOI": "AOI boundary",                  # legend label for AOI boundary
    "LEG_AOI_LW": 1.2,                          # legend line width for AOI item

    # ---- Route style ----
    "ROUTE_COLOR": "red",                       # route line color
    "ROUTE_LW": 2.6,                            # route line width
    "ROUTE_ALPHA": 1.0,                         # route transparency
    "LBL_ROUTE": "Route",                       # legend label for route

    # ---- Start/Destination markers ----
    "START_COLOR": "#1b9e77",                   # start marker color
    "START_SIZE": 48,                           # start marker size
    "DEST_COLOR": "black",                      # destination marker color
    "DEST_SIZE": 40,                            # destination marker size
    "LBL_START": "Start",                       # legend label for start
    "LBL_DEST": "Destination",                  # legend label for destination
    "LEG_MARKER_SIZE": 8,                       # legend marker size for start/dest items

    # ---- North arrow ----
    "ADD_NORTH_ARROW": True,                    # toggle north arrow
    "NA_X": 0.05, "NA_Y": 0.15,                 # arrow position (axes fraction)
    "NA_LEN": 0.08,                             # arrow length (axes fraction)
    "NA_LABEL": "N",                            # arrow label
    "NA_COLOR": "black", "NA_LW": 2, "NA_FONTSIZE": 14,  # arrow color/line width/font size

    # ---- Scalebar ----
    "ADD_SCALEBAR": True,                       # toggle scalebar
    "SB_UNITS": "m",                            # scalebar units
    "SB_LOC": "lower right",                    # scalebar location
    "SB_BOX_ALPHA": 0.8,                        # scalebar box transparency

    # ---- Title ----
    "TITLE_PREFIX": "Shortest Path"             # title prefix → "<prefix> i — length = Xm"
}
# ------------------------------------------------------------------------------

def _route_to_gdfs_ll(G, route):
    res = ox.routing.route_to_gdf(G, route, weight="length")
    if isinstance(res, tuple):
        return res[0], res[1]
    xs = [G.nodes[n]["x"] for n in route]
    ys = [G.nodes[n]["y"] for n in route]
    nodes_ll = gpd.GeoDataFrame({"node": route, "seq": range(len(route))},
                                geometry=gpd.points_from_xy(xs, ys), crs=4326)
    return res, nodes_ll

# Base network in meters (CRS from edges_ok)
base_utm = edges_ok.estimate_utm_crs()
base_m   = edges_ok.to_crs(base_utm)

# Optional AOI (in same meters CRS)
aoi_m = aoi_gdf_ll.to_crs(base_utm) if P["DRAW_AOI"] and 'aoi_gdf_ll' in globals() else None

# --- Save combined CSV for all routes (unchanged methodology) ---
all_rows = []
for i, r in enumerate(routes_list[:ROUTE_TARGET]):
    for row in export_nodes_csv_rows(G, r):
        row["route_id"] = i
        all_rows.append(row)

csv_path = Path(SP_OUT_DIR) / P["CSV_NAME"]
pd.DataFrame(all_rows).to_csv(csv_path, index=False)
print(f" Saved CSV → {csv_path}")

# --- Plot & save (smaller images; one PNG per route) ---
def _plot_route_png(i, r):
    edges_ll, nodes_ll = _route_to_gdfs_ll(G, r)
    edges_m = edges_ll.to_crs(base_utm)
    nodes_m = nodes_ll.to_crs(base_utm)

    # Start/End markers (project to meters CRS)
    start_lon, start_lat = G.nodes[orig_node]["x"], G.nodes[orig_node]["y"]
    dest_lon,  dest_lat  = G.nodes[dest_node]["x"],  G.nodes[dest_node]["y"]
    start_m = gpd.GeoSeries([Point(start_lon, start_lat)], crs=4326).to_crs(base_utm)
    dest_m  = gpd.GeoSeries([Point(dest_lon,  dest_lat)],  crs=4326).to_crs(base_utm)

    # Frame (zoom if user provided bbox; else auto from layers)
    if all(P[k] is not None for k in ("lon_min","lon_max","lat_min","lat_max")):
        zoom_ll = gpd.GeoDataFrame(
            geometry=[box(min(P["lon_min"], P["lon_max"]),
                          min(P["lat_min"], P["lat_max"]),
                          max(P["lon_min"], P["lon_max"]),
                          max(P["lat_min"], P["lat_max"]))],
            crs=4326
        ).to_crs(base_utm)
        xmin, ymin, xmax, ymax = zoom_ll.total_bounds
    else:
        bounds = []
        if not base_m.empty:  bounds.append(base_m.total_bounds)
        if not edges_m.empty: bounds.append(edges_m.total_bounds)
        if aoi_m is not None and not aoi_m.empty: bounds.append(aoi_m.total_bounds)
        xmin = min(b[0] for b in bounds); ymin = min(b[1] for b in bounds)
        xmax = max(b[2] for b in bounds); ymax = max(b[3] for b in bounds)

    # Plot
    fig, ax = plt.subplots(figsize=P["FIGSIZE"])
    if not base_m.empty:
        base_m.plot(ax=ax, color=P["BASE_COLOR"], linewidth=P["BASE_LW"],
                    alpha=P["BASE_ALPHA"], zorder=1)
    if aoi_m is not None and not aoi_m.empty:
        aoi_m.boundary.plot(ax=ax, color=P["AOI_COLOR"], linewidth=P["AOI_LW"],
                            alpha=P["AOI_ALPHA"], zorder=2)

    edges_m.plot(ax=ax, color=P["ROUTE_COLOR"], linewidth=P["ROUTE_LW"],
                 alpha=P["ROUTE_ALPHA"], zorder=3)
    if len(nodes_m) >= 2:
        nodes_m.iloc[[0, -1]].plot(ax=ax, color=P["ROUTE_COLOR"],
                                   markersize=max(P["ROUTE_LW"]*5, 10), zorder=4)

    start_m.plot(ax=ax, color=P["START_COLOR"], markersize=P["START_SIZE"], marker="o", zorder=5)
    dest_m.plot(ax=ax,  color=P["DEST_COLOR"],  markersize=P["DEST_SIZE"],  marker="o", zorder=5)

    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax)

    # North arrow
    if P["ADD_NORTH_ARROW"]:
        ax.annotate(P["NA_LABEL"], xy=(P["NA_X"], P["NA_Y"]), xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                    xycoords="axes fraction", textcoords="axes fraction",
                    ha="center", va="center", fontsize=P["NA_FONTSIZE"], fontweight="bold",
                    arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                    clip_on=False, zorder=20)

    # Scalebar
    if P["ADD_SCALEBAR"]:
        ax.add_artist(ScaleBar(1, P["SB_UNITS"], location=P["SB_LOC"], box_alpha=P["SB_BOX_ALPHA"]))

    ax.set_aspect("equal"); ax.ticklabel_format(style="plain")
    ax.set_xlabel(P["XLABEL"]); ax.set_ylabel(P["YLABEL"])
    length_m = get_route_length(G, r)
    ax.set_title(f"{P['TITLE_PREFIX']} {i} — length = {length_m:.1f} m")

    legend_items = [
        Line2D([0],[0], color=P["BASE_COLOR"], lw=3, label=P["LBL_BASE"]),
        Line2D([0],[0], color=P["ROUTE_COLOR"], lw=3, label=P["LBL_ROUTE"]),
        Line2D([0],[0], marker="o", color="none", markerfacecolor=P["START_COLOR"],
               markersize=P["LEG_MARKER_SIZE"], label=P["LBL_START"]),
        Line2D([0],[0], marker="o", color="none", markerfacecolor=P["DEST_COLOR"],
               markersize=P["LEG_MARKER_SIZE"], label=P["LBL_DEST"]),
        Line2D([0],[0], color=P["AOI_COLOR"], lw=P["LEG_AOI_LW"], label=P["LBL_AOI"])
    ]
    ax.legend(handles=legend_items, loc=P["LEGEND_LOC"], frameon=True)

    out_png_i = Path(SP_OUT_DIR) / P["OUT_NAME_FMT"].format(i=i)
    fig.savefig(out_png_i, dpi=P["DPI"], bbox_inches="tight")
    plt.close(fig)
    print(f" Saved: {out_png_i}")
    display(Image(filename=str(out_png_i)))

# Render all requested routes
for i, r in enumerate(routes_list[:ROUTE_TARGET]):
    _plot_route_png(i, r)


## I) Utility 2: Betweenness Centrality

In [ ]:
# ========================= I0) Imports  =========================
import os
import math
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap, ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D

from shapely.geometry import Point, box
from IPython.display import Image, display
from matplotlib_scalebar.scalebar import ScaleBar
import yaml
from math import radians, sin, asin, sqrt, cos
from matplotlib import colormaps

In [ ]:
# ====================== I1) User Options & Paths ======================
# --- Spatial subset for BC (lon/lat degrees). Set USE_BBOX=False to use full area in configuration
USE_BBOX = require("USE_BBOX", bool)
lon_min, lon_max = require("lon_min", float), require("lon_max", float)
lat_min, lat_max = require("lat_min", float), require("lat_max", float)

# --- Betweenness centrality options ---
USE_APPROX_BC = require("USE_APPROX_BC", bool)     # True: randomized approx (fast), False: exact (slow)
K_SAMPLES     = require("K_SAMPLES", int)          # used only if USE_APPROX_BC=True (clipped to #nodes)
BC_WEIGHT     = require("BC_WEIGHT", str)          # edge weight for BC
BC_NORMALIZED = require("BC_NORMALIZED", bool)     # normalize BC values to [0,1] scale

# --- Outputs ---
OUT_BASE   = r"./Outputs"
A_OUT_DIR  = os.path.join(OUT_BASE, "A")
E_OUT_DIR  = os.path.join(OUT_BASE, "E")
F_OUT_DIR  = os.path.join(OUT_BASE, "F")
I_OUT_DIR  = os.path.join(OUT_BASE, "I")
IMAGES_DIR = os.path.join(OUT_BASE, "Images")
os.makedirs(I_OUT_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)

try:
    _have_scalebar = True
except Exception:
    _have_scalebar = False


In [ ]:
# ============== I2) Gather UNDAMAGED nodes/edges ===============
nodes_ok_src = globals().get("nodes_ok", None)
edges_ok_src = globals().get("edges_ok", None)

if (nodes_ok_src is None or getattr(nodes_ok_src, "empty", True)) and ("nodes_eval" in globals()):
    n_ = globals()["nodes_eval"]
    if isinstance(n_, gpd.GeoDataFrame) and not n_.empty and ("Damaged" in n_.columns):
        nodes_ok_src = n_[n_["Damaged"].astype(str).str.lower().eq("no")].copy()

if (edges_ok_src is None or getattr(edges_ok_src, "empty", True)) and ("edges_eval" in globals()):
    e_ = globals()["edges_eval"]
    if isinstance(e_, gpd.GeoDataFrame) and not e_.empty and ("Damaged" in e_.columns):
        edges_ok_src = e_[e_["Damaged"].astype(str).str.lower().eq("no")].copy()

if (edges_ok_src is None or getattr(edges_ok_src, "empty", True)):
    cand_edges = [
        Path(F_OUT_DIR) / "Evaluated_Edges_withTunnels_Bridges.shp",
        Path(E_OUT_DIR) / "Evaluated_Edges.shp",
        Path(A_OUT_DIR) / "osm_edges.shp",
    ]
    for p in cand_edges:
        if p.exists():
            g = gpd.read_file(p)
            if "Damaged" in g.columns:
                edges_ok_src = g[g["Damaged"].astype(str).str.lower().eq("no")].copy()
            else:
                edges_ok_src = g.copy()
            if not edges_ok_src.empty:
                break

if (nodes_ok_src is None or getattr(nodes_ok_src, "empty", True)):
    cand_nodes = [
        Path(F_OUT_DIR) / "Evaluated_Nodes_withTunnels_Bridges.shp",
        Path(A_OUT_DIR) / "osm_nodes.shp",
    ]
    for p in cand_nodes:
        if p.exists():
            n = gpd.read_file(p)
            if "Damaged" in n.columns:
                nodes_ok_src = n[n["Damaged"].astype(str).str.lower().eq("no")].copy()
            else:
                nodes_ok_src = n.copy()
            if not nodes_ok_src.empty:
                break

if nodes_ok_src is None or edges_ok_src is None or nodes_ok_src.empty or edges_ok_src.empty:
    raise RuntimeError("Could not assemble undamaged nodes/edges for BC.")

print(f"UNDAMAGED for BC → nodes: {len(nodes_ok_src):,} | edges: {len(edges_ok_src):,}")

In [ ]:
# ================= I3) Normalize schema helpers =================
def ensure_nodes_index_xy(nodes_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Ensure:
      - index is 'osmid' (if present),
      - CRS is EPSG:4326,
      - columns 'x' and 'y' are lon/lat in degrees from geometry.
    """
    n = nodes_gdf.copy()

    # --- Index: use OSMID if present ---
    if "osmid" in n.columns:
        if n.index.name != "osmid":
            n = n.set_index("osmid", drop=False)
    else:
        # fallback: create an osmid from the index
        if n.index.name != "osmid":
            n = n.reset_index().rename(columns={"index": "osmid"})
            n = n.set_index("osmid", drop=False)

    # --- CRS: force WGS84 lon/lat ---
    if n.crs is None:
        n = n.set_crs(4326, allow_override=True)
    if n.crs.to_epsg() != 4326:
        n = n.to_crs(4326)

    # --- x,y: always from geometry in EPSG:4326 ---
    n["x"] = n.geometry.x  # longitude
    n["y"] = n.geometry.y  # latitude

    return n

def _ensure_edges_uvk(edges_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Keep u/v/key as columns; do not set MultiIndex here."""
    e = edges_gdf.copy()
    if isinstance(e.index, pd.MultiIndex):
        e = e.reset_index()
        idx_names = list(edges_gdf.index.names)
        rename_map = {}
        if len(idx_names) >= 1 and idx_names[0] and idx_names[0] != 'u':   rename_map[idx_names[0]] = 'u'
        if len(idx_names) >= 2 and idx_names[1] and idx_names[1] != 'v':   rename_map[idx_names[1]] = 'v'
        if len(idx_names) >= 3 and idx_names[2] and idx_names[2] != 'key': rename_map[idx_names[2]] = 'key'
        if rename_map:
            e = e.rename(columns=rename_map)
    missing_uv = [c for c in ('u','v') if c not in e.columns]
    if missing_uv:
        raise ValueError(f"Edges must have columns 'u' and 'v'; missing: {missing_uv}")
    if 'key' not in e.columns:
        e['key'] = e.groupby(['u','v']).cumcount()
    return e

def _ensure_lengths(edges_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Ensure a 'length' column in meters (prefer projected UTM; fallback haversine)."""
    e = edges_gdf.copy()
    if "length" in e.columns and e["length"].notna().all():
        return e
    try:
        utm = e.estimate_utm_crs()
        e["length"] = e.to_crs(utm).geometry.length
    except Exception:
        def hav_m(geom):
            try:
                coords = list(geom.coords)
            except Exception:
                return 0.0
            R = 6371008.8
            s = 0.0
            for (x1,y1),(x2,y2) in zip(coords[:-1], coords[1:]):
                dlat = radians(y2-y1); dlon = radians(x2-x1)
                a = sin(dlat/2)**2 + cos(radians(y1))*cos(radians(y2))*sin(dlon/2)**2
                s += 2*R*asin(sqrt(a))
            return s
        e["length"] = e.geometry.apply(hav_m)
    return e

# Normalize + dtype align
nodes_bc = ensure_nodes_index_xy(nodes_ok_src)
edges_bc = _ensure_edges_uvk(edges_ok_src)

# dtype align node index with edge u/v dtype (no MultiIndex yet)
u_dtype = edges_bc['u'].dtype
if nodes_bc.index.dtype != u_dtype:
    nodes_bc.index = nodes_bc.index.astype(u_dtype)
    if "osmid" in nodes_bc.columns:
        nodes_bc["osmid"] = nodes_bc["osmid"].astype(u_dtype)

if nodes_bc.crs is None:
    nodes_bc = nodes_bc.set_crs(4326, allow_override=True)
if edges_bc.crs is None:
    edges_bc = edges_bc.set_crs(nodes_bc.crs, allow_override=True)

print(" Schema OK for OSMnx (pre-clip)")
print("  Nodes index:", nodes_bc.index.name, "| dtype:", nodes_bc.index.dtype)
print("  Edges u/v exist:", set(['u','v']).issubset(edges_bc.columns))

In [ ]:
# ========================= I4) Optional lon/lat bbox clip =========================
def _valid(x): return x is not None and isinstance(x, (int, float))
use_bbox = USE_BBOX and all(_valid(v) for v in [lon_min, lon_max, lat_min, lat_max])

if use_bbox:
    nodes_ll = nodes_bc.to_crs(4326).copy()
    edges_ll = edges_bc.to_crs(4326).copy()

    xmin, xmax = min(lon_min, lon_max), max(lon_min, lon_max)
    ymin, ymax = min(lat_min, lat_max), max(lat_min, lat_max)
    rect = box(xmin, ymin, xmax, ymax)

    # keep nodes strictly inside bbox
    nodes_clip = nodes_ll[nodes_ll.geometry.within(rect)].copy()
    keep_ids = set(nodes_clip["osmid"].tolist())

    # keep edges whose BOTH endpoints are inside
    if not {"u", "v"}.issubset(edges_ll.columns):
        raise ValueError("Edges must expose 'u' and 'v' columns at this step.")
    edges_clip = edges_ll[edges_ll["u"].isin(keep_ids) & edges_ll["v"].isin(keep_ids)].copy()

    # back to original CRS
    nodes_use = nodes_clip.to_crs(nodes_bc.crs)
    edges_use = edges_clip.to_crs(edges_bc.crs)

    print(f" BBox filter applied → nodes: {len(nodes_use):,} | edges: {len(edges_use):,}")
else:
    nodes_use = nodes_bc
    edges_use = edges_bc
    print(f" Using full undamaged network → nodes: {len(nodes_use):,} | edges: {len(edges_use):,}")

if nodes_use.empty or edges_use.empty:
    raise SystemExit("No nodes/edges in the selected extent for BC.")

# Compute edge lengths after final selection
edges_use = _ensure_lengths(edges_use)

In [ ]:
# ========================= I5) Finalize OSMnx-friendly indices =========================
# --- Nodes: ensure index=osmid and x/y present ---
nodes_g = nodes_use.copy()
if 'osmid' not in nodes_g.columns:
    nodes_g = nodes_g.reset_index().rename(columns={'index': 'osmid'})
if nodes_g.index.name != 'osmid':
    nodes_g = nodes_g.set_index('osmid', drop=False)

if ('x' not in nodes_g.columns) or ('y' not in nodes_g.columns):
    if nodes_g.crs and getattr(nodes_g.crs, "is_projected", False):
        _n_ll = nodes_g.to_crs(4326)
        nodes_g['x'] = _n_ll.geometry.x
        nodes_g['y'] = _n_ll.geometry.y
    else:
        nodes_g['x'] = nodes_g.geometry.x
        nodes_g['y'] = nodes_g.geometry.y

# --- Edges: u/v/key are already columns; create canonical MultiIndex once for OSMnx ---
edges_g = edges_use.copy()
missing_uv = [c for c in ('u','v') if c not in edges_g.columns]
if missing_uv:
    raise KeyError(f"Edges must have 'u' and 'v' columns; missing: {missing_uv}")

# ensure 'key' exists
if 'key' not in edges_g.columns:
    edges_g['key'] = edges_g.groupby(['u','v']).cumcount()

# set MultiIndex and DROP u/v/key from columns to avoid duplication
edges_g = edges_g.set_index(['u', 'v', 'key'], drop=True)


In [ ]:
# ========================= I6) Build graph → undirected → largest CC =========================
# Ensure compatible CRS tags
if nodes_g.crs is None:
    nodes_g = nodes_g.set_crs(4326, allow_override=True)
if edges_g.crs is None:
    edges_g = edges_g.set_crs(nodes_g.crs, allow_override=True)

# Build MultiDiGraph from GDFs
G_multi = ox.convert.graph_from_gdfs(nodes_g, edges_g)

# Make undirected weighted by 'length'
G_und = ox.convert.to_undirected(G_multi)

# Largest connected component for stable BC
if G_und.number_of_nodes() == 0:
    raise SystemExit("Graph has no nodes after preprocessing.")
largest_cc = max(nx.connected_components(G_und), key=len)
G = G_und.subgraph(largest_cc).copy()

print(f"Graph for BC → nodes: {G.number_of_nodes():,} | edges: {G.number_of_edges():,}")

In [ ]:
# ========================= I7) Betweenness centrality =========================
if USE_APPROX_BC:
    k = min(K_SAMPLES, G.number_of_nodes())
    bc_dict = nx.betweenness_centrality(G, k=k, seed=0, weight=BC_WEIGHT, normalized=BC_NORMALIZED)
    print(f"Computed APPROX betweenness centrality with k={k}.")
else:
    bc_dict = nx.betweenness_centrality(G, weight=BC_WEIGHT, normalized=BC_NORMALIZED)
    print("Computed EXACT betweenness centrality.")

# Quick peek
if bc_dict:
    top_node = max(bc_dict, key=bc_dict.get)
    print(f" Top BC node: {top_node} | BC={bc_dict[top_node]:.6f}")
else:
    print(" Empty BC result (degenerate component?).")

In [ ]:
# ========================= I8) Attach BC back to nodes dataset =========================
nodes_bc = nodes_bc.copy()
if 'osmid' not in nodes_bc.columns:
    nodes_bc = nodes_bc.reset_index().rename(columns={'index': 'osmid'}).set_index('osmid', drop=False)

bc_series = pd.Series(bc_dict, name='bc', dtype=float)
nodes_bc['bc'] = nodes_bc['osmid'].map(bc_series).fillna(0.0).astype(float)

print(f"nodes_bc w/ BC assigned: {len(nodes_bc):,} rows | bc>0 count: {(nodes_bc['bc']>0).sum():,}")

In [ ]:
# ========================= I9) Plot BC map =========================
P = {
    # ---- Output ----
    "OUT_NAME": "Betweenness_Centrality_Map.png",   # filename for the exported PNG

    # ---- Data / threshold ----
    "BC_THRESHOLD": 0.1,                           # show nodes whose betweenness centrality >= this value

    # ---- Manual zoom in EPSG:4326 (set all to None for auto extent) ----
    "lon_min": 36.900,             # min longitude of custom zoom (None = auto)
    "lon_max": 36.930,             # max longitude (None = auto)
    "lat_min": 37.570,             # min latitude  (None = auto)
    "lat_max": 37.600,             # max latitude  (None = auto)

    # ---- Figure & theme ----
    "FIGSIZE": (8, 7),                              # figure size (width, height) in inches
    "DPI": 300,                                     # export resolution (dots per inch)
    "BG_COLOR": "black",                            # background color for figure/axes
    "FG_COLOR": "white",                            # foreground color for ticks/labels/spines

    # ---- Base layers ----
    "EDGE_COLOR": "#808080",                        # line color for the base network edges
    "EDGE_LW": 0.7,                                 # linewidth for base network edges
    "EDGE_ALPHA": 1.0,                              # transparency for base network edges (0..1)
    "AOI_COLOR": "white",                           # color for AOI boundary
    "AOI_LW": 1.2,                                  # linewidth for AOI boundary
    "AOI_ALPHA": 1.0,                               # transparency for AOI boundary (0..1)
    "NODE_SIZE": 12,                                # marker size for plotted nodes (above threshold)
    "NODE_EDGEWIDTH": 0,                            # outline width for node markers (0 = none)
    "CMAP": "plasma",                               # colormap used to color nodes by BC value

    # ---- Colorbar ----
    "ADD_COLORBAR": True,                           # toggle colorbar visibility
    "CB_LABEL": "Betweenness Centrality",           # colorbar label text
    "CB_FRACTION": 0.03,                            # colorbar size fraction relative to axes
    "CB_PAD": 0.02,                                 # padding between axes and colorbar
    "CB_LABEL_COLOR": "white",                      # color for colorbar label text
    "CB_TICK_COLOR": "white",                       # color for colorbar tick labels

    # ---- Axes labels & title ----
    "XLABEL": "Easting (m)",                        # x-axis label
    "YLABEL": "Northing (m)",                       # y-axis label
    "TITLE": "Betweenness Centrality",              # plot title text
    "TITLE_COLOR": "white",                         # plot title color

    # ---- North arrow ----
    "ADD_NORTH_ARROW": True,                        # toggle north arrow
    "NA_X": 0.05, "NA_Y": 0.15,                     # arrow position in axes fraction (0..1)
    "NA_LEN": 0.08,                                 # arrow length in axes fraction
    "NA_LABEL": "N",                                # text label shown at arrow head
    "NA_COLOR": "white", "NA_LW": 2, "NA_FONTSIZE": 14,  # arrow color, line width, font size

    # ---- Scalebar ----
    "ADD_SCALEBAR": True,                           # toggle scalebar
    "SB_UNITS": "m",                                # scalebar units label
    "SB_LOC": "lower right",                        # scalebar location on axes
    "SB_BOX_ALPHA": 0.8,                            # scalebar box transparency (0..1)
    "SB_COLOR": "black",                            # scalebar text/line color

    # ---- Legend ----
    "ADD_LEGEND": True,                             # toggle legend visibility
    "LEGEND_LOC": "upper right",                    # legend placement
    "LEGEND_FACE": "white",                         # legend box facecolor
    "LEGEND_EDGE": "black",                         # legend box edgecolor
    "LEGEND_FRAME": True,                           # draw legend frame box
    "LBL_EDGES": "Undamaged edges",                 # legend label: base network line
    "LBL_NODES": "Nodes (BC ≥ {thr:.2f})",          # legend label: BC nodes (format with BC_THRESHOLD)
    "LBL_AOI": "AOI boundary",                      # legend label: AOI boundary
    "LEG_EDGES_LW": 2.0,                            # legend line width for edges item
    "LEG_AOI_LW": 1.2,                              # legend line width for AOI item
    "LEG_NODE_MARKERSIZE": 7                        # legend marker size for node item
}

# ---------- Prepare projected layers (UTM) ----------
seed = edges_use if not edges_use.empty else nodes_use
utm_crs = seed.to_crs(4326).estimate_utm_crs()

edges_m = edges_use.to_crs(utm_crs)
nodes_m = nodes_bc.to_crs(utm_crs)   # full nodes (bc added), for plotting thresholded points
aoi_m   = aoi_gdf_ll.to_crs(utm_crs) if 'aoi_gdf_ll' in globals() else None

# Threshold + color scale
max_bc_val = float(nodes_m["bc"].max()) if len(nodes_m) else 0.0
cmap = get_cmap(P["CMAP"])
norm = Normalize(vmin=0.0, vmax=max_bc_val if max_bc_val > 0 else 1.0)
nodes_thr = nodes_m[nodes_m["bc"] >= P["BC_THRESHOLD"]].copy()
if nodes_thr.empty:
    print(f"Warning: no nodes with BC ≥ {P['BC_THRESHOLD']}. Consider lowering the threshold.")

# Compute extent (auto-fit to drawn layers, with optional manual zoom in lon/lat)
bounds = []
if not edges_m.empty:
    bounds.append(edges_m.total_bounds)
if not nodes_m.empty:
    bounds.append(nodes_m.total_bounds)
if aoi_m is not None and not aoi_m.empty:
    bounds.append(aoi_m.total_bounds)

auto_xmin = min(b[0] for b in bounds)
auto_ymin = min(b[1] for b in bounds)
auto_xmax = max(b[2] for b in bounds)
auto_ymax = max(b[3] for b in bounds)

lon_min = P.get("lon_min", None)
lon_max = P.get("lon_max", None)
lat_min = P.get("lat_min", None)
lat_max = P.get("lat_max", None)

# If all four lon/lat limits are provided, use them as a manual zoom box
if (lon_min is not None) and (lon_max is not None) and (lat_min is not None) and (lat_max is not None):
    zoom_box_ll = box(lon_min, lat_min, lon_max, lat_max)
    zoom_box_utm = gpd.GeoSeries([zoom_box_ll], crs="EPSG:4326").to_crs(utm_crs).total_bounds
    xmin, ymin, xmax, ymax = zoom_box_utm
else:
    xmin, ymin, xmax, ymax = auto_xmin, auto_ymin, auto_xmax, auto_ymax

# --- Plot ---
fig, ax = plt.subplots(figsize=P["FIGSIZE"], facecolor=P["BG_COLOR"])
ax.set_facecolor(P["BG_COLOR"])

# Base network & AOI
if not edges_m.empty:
    edges_m.plot(ax=ax,
                 color=P["EDGE_COLOR"],
                 linewidth=P["EDGE_LW"],
                 alpha=P["EDGE_ALPHA"],
                 zorder=1)

if aoi_m is not None and not aoi_m.empty:
    aoi_m.boundary.plot(ax=ax,
                        color=P["AOI_COLOR"],
                        linewidth=P["AOI_LW"],
                        alpha=P["AOI_ALPHA"],
                        zorder=2)

# Nodes (BC ≥ threshold)
if not nodes_thr.empty:
    ax.scatter(nodes_thr.geometry.x, nodes_thr.geometry.y,
               c=nodes_thr["bc"],
               cmap=cmap,
               norm=norm,
               s=P["NODE_SIZE"],
               linewidths=P["NODE_EDGEWIDTH"],
               zorder=3)

# Colorbar
if P["ADD_COLORBAR"]:
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax,
                        fraction=P["CB_FRACTION"],
                        pad=P["CB_PAD"])
    cbar.set_label(P["CB_LABEL"], color=P["CB_LABEL_COLOR"])
    cbar.ax.yaxis.set_tick_params(color=P["CB_TICK_COLOR"])
    plt.setp(cbar.ax.get_yticklabels(), color=P["CB_TICK_COLOR"])

# Frame + decor
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

if P["ADD_NORTH_ARROW"]:
    ax.annotate(P["NA_LABEL"],
                xy=(P["NA_X"], P["NA_Y"]),
                xytext=(P["NA_X"], P["NA_Y"] - P["NA_LEN"]),
                xycoords="axes fraction",
                textcoords="axes fraction",
                ha="center",
                va="center",
                fontsize=P["NA_FONTSIZE"],
                fontweight="bold",
                arrowprops=dict(arrowstyle="-|>", lw=P["NA_LW"], color=P["NA_COLOR"]),
                color=P["NA_COLOR"],
                clip_on=False,
                zorder=20)

# Scalebar
if P["ADD_SCALEBAR"]:
    if _have_scalebar:
        ax.add_artist(
            ScaleBar(1,
                     P["SB_UNITS"],
                     location=P["SB_LOC"],
                     box_alpha=P["SB_BOX_ALPHA"],
                     color=P["SB_COLOR"])
        )
    else:
        print("Note: scalebar requested but matplotlib-scalebar is not installed. Skipping.")

ax.set_aspect("equal")
ax.ticklabel_format(style="plain")
ax.tick_params(colors=P["FG_COLOR"])
for sp in ax.spines.values():
    sp.set_edgecolor(P["FG_COLOR"])
ax.set_xlabel(P["XLABEL"], color=P["FG_COLOR"])
ax.set_ylabel(P["YLABEL"], color=P["FG_COLOR"])
ax.set_title(P["TITLE"], color=P["TITLE_COLOR"])

# Legend
if P["ADD_LEGEND"]:
    legend_items = [
        Line2D([0], [0],
               color=P["EDGE_COLOR"],
               lw=P["LEG_EDGES_LW"],
               label=P["LBL_EDGES"]),
        Line2D([0], [0],
               marker="o",
               color="none",
               markerfacecolor=P["FG_COLOR"],
               markersize=P["LEG_NODE_MARKERSIZE"],
               label=P["LBL_NODES"].format(thr=P["BC_THRESHOLD"])),
        Line2D([0], [0],
               color=P["AOI_COLOR"],
               lw=P["LEG_AOI_LW"],
               label=P["LBL_AOI"]),
    ]
    leg = ax.legend(handles=legend_items,
                    loc=P["LEGEND_LOC"],
                    frameon=P["LEGEND_FRAME"],
                    facecolor=P["LEGEND_FACE"],
                    edgecolor=P["LEGEND_EDGE"])
    leg.set_zorder(1000)

# Save + show
out_path = Path(IMAGES_DIR) / P["OUT_NAME"]
fig.savefig(out_path,
            dpi=P["DPI"],
            bbox_inches="tight",
            facecolor=fig.get_facecolor())
plt.close(fig)
print(f" Saved: {out_path}")
display(Image(filename=out_path))


In [ ]:
# ========================= I10) Export BC table (node id, lon, lat, BC) =========================
# Ensure required columns and WGS84 coordinates
_nodes = nodes_bc.copy()
if 'osmid' not in _nodes.columns:
    _nodes = _nodes.reset_index().rename(columns={'index':'osmid'})

nodes_ll = _nodes.to_crs(4326) if getattr(_nodes.crs, "to_epsg", lambda: None)() != 4326 else _nodes

bc_df = pd.DataFrame({
    "node_id": nodes_ll["osmid"].astype(object),
    "lon": nodes_ll.geometry.x.astype(float),
    "lat": nodes_ll.geometry.y.astype(float),
    "BC": nodes_ll["bc"].astype(float)
}).sort_values("BC", ascending=False)

# Save CSV to Outputs/I
bc_csv = Path(I_OUT_DIR) / "Betweenness_Centrality_nodes.csv"
bc_df.to_csv(bc_csv, index=False)
print(f" Saved BC CSV → {bc_csv}")
try:
    display(bc_df.head(10))
except Exception:
    pass